In [ ]:
# =========================== qc_forest2.py ============================
"""
Overall data health (Level-1 MWIR) — FOREST-2 + Side-by-Side Visuals
--------------------------------------------------------------------
QC metrics for a FOREST-2 MWIR Level-1 GeoTIFF + inline previews:

 • QC table: Metric | Value | Description
     • Figures:
 • Left  panel = Sobel gradient magnitude (p99-stretched, with colorbar; title shows p50/p95/p99)
 • Right panel = Radiance (grayscale) with Hot pixels (red) and Cold pixels (blue), with grayscale colorbar

"""

# ============================== Imports ===============================

from pathlib import Path
import math
from typing import Tuple, Dict, Any

import numpy as np
import pandas as pd
import rasterio
from scipy import ndimage as ndi
import matplotlib.pyplot as plt


# =========================== Configuration ============================

# Additional NoData values commonly encountered in L1 MWIR GeoTIFFs
NO_DATA_CANDIDATES = (-3.40282e38, 65555, 65535)


# ============================== Utilities =============================

def build_valid_mask(arr: np.ndarray, nodata_tag) -> np.ndarray:
    """
    Construct a boolean mask of valid pixels.

    Rules:
      • Valid if finite
      • Excludes declared nodata_tag (if present and not NaN)
      • Excludes common values in NO_DATA_CANDIDATES
      • Excludes extreme negative fillers (≤ -1e30)

    Parameters
    ----------
    arr : np.ndarray
        Input radiance array.
    nodata_tag : numeric or None
        NoData value declared in the raster metadata.

    Returns
    -------
    np.ndarray (bool)
        True for valid pixels, False otherwise.
    """
    mask = np.isfinite(arr)
    if nodata_tag is not None and not (isinstance(nodata_tag, float) and math.isnan(nodata_tag)):
        mask &= ~(arr == nodata_tag)
    for v in NO_DATA_CANDIDATES:
        mask &= ~(arr == v)
    mask &= ~(arr <= -1e30)
    return mask


def robust_percentiles(x: np.ndarray, ps=(2, 98)):
    """Return robust percentiles of a 1D array; NaN-safe for empty inputs."""
    if x.size == 0:
        return [np.nan for _ in ps]
    return [float(np.percentile(x, p)) for p in ps]


def zscore_1d(x: np.ndarray) -> np.ndarray:
    """Compute 1D z-scores with NaN-safe std handling."""
    m, s = np.nanmean(x), np.nanstd(x)
    if not np.isfinite(s) or s == 0:
        return (x - m)
    return (x - m) / s


def count_internal_holes(nodata_mask: np.ndarray) -> int:
    """
    Count connected NoData components that do NOT touch the image border.
    """
    lab, ncomp = ndi.label(nodata_mask)
    if ncomp == 0:
        return 0
    h, w = nodata_mask.shape
    touches = set()
    touches.update(np.unique(lab[0, :]))
    touches.update(np.unique(lab[-1, :]))
    touches.update(np.unique(lab[:, 0]))
    touches.update(np.unique(lab[:, -1]))
    touches.discard(0)
    holes = sum((i not in touches) for i in range(1, ncomp + 1))
    return int(holes)


def internal_holes_mask_and_sizes(nodata_mask: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Identify interior NoData components and return:
      • holes_mask: boolean mask of interior NoData pixels
      • hole_sizes: array of pixel counts per interior component
    """
    lab, ncomp = ndi.label(nodata_mask)
    if ncomp == 0:
        return np.zeros_like(nodata_mask, dtype=bool), np.array([], dtype=int)
    h, w = nodata_mask.shape
    border_labels = set(np.unique(lab[0, :])) | set(np.unique(lab[-1, :])) \
                    | set(np.unique(lab[:, 0])) | set(np.unique(lab[:, -1]))
    border_labels.discard(0)
    holes_mask = nodata_mask & ~np.isin(lab, list(border_labels))
    counts = np.bincount(lab.ravel())
    hole_labels = np.setdiff1d(np.arange(1, counts.size),
                               np.fromiter(border_labels, dtype=int, count=len(border_labels)))
    hole_sizes = counts[hole_labels] if hole_labels.size else np.array([], dtype=int)
    return holes_mask, hole_sizes


def gradient_percentiles(arr: np.ndarray, mask_valid: np.ndarray) -> Dict[str, float]:
    """
    Compute Sobel gradient magnitude over valid pixels and return percentiles.

    Returns
    -------
    dict
        {'grad_p50', 'grad_p95', 'grad_p99'}
    """
    filled = np.where(mask_valid, arr, np.nan)
    filled = np.nan_to_num(filled, nan=0.0)
    gx = ndi.sobel(filled, axis=1)
    gy = ndi.sobel(filled, axis=0)
    gm = np.hypot(gx, gy)
    vals = gm[mask_valid]
    if vals.size == 0:
        return dict(grad_p50=np.nan, grad_p95=np.nan, grad_p99=np.nan)
    return dict(
        grad_p50=float(np.percentile(vals, 50)),
        grad_p95=float(np.percentile(vals, 95)),
        grad_p99=float(np.percentile(vals, 99)),
    )


# ====================== Gradient & Hot/Cold Layers =====================

def compute_sobel_magnitude(arr: np.ndarray, mask_valid: np.ndarray) -> np.ndarray:
    """
    Sobel gradient magnitude (float32), masked to NaN outside valid pixels.
    """
    safe = np.where(mask_valid, arr, np.nan)
    safe = np.nan_to_num(safe, nan=0.0)
    gx = ndi.sobel(safe, axis=1)
    gy = ndi.sobel(safe, axis=0)
    gm = np.hypot(gx, gy).astype(np.float32)
    gm[~mask_valid] = np.nan
    return gm


def compute_hot_cold_mask(arr: np.ndarray, mask_valid: np.ndarray, z_thresh: float = 6.0) -> np.ndarray:
    """
    Flag hot/cold outliers relative to valid-pixel mean/std.

    Returns
    -------
    np.ndarray (int)
        1 for hot (> +z_thresh), -1 for cold (< -z_thresh), 0 otherwise.
    """
    vals = arr[mask_valid]
    if vals.size == 0:
        return np.zeros_like(arr, dtype=int)
    m, s = np.mean(vals), np.std(vals)
    s = s if s != 0 else 1.0
    z = (arr - m) / s
    hot_mask = (z > z_thresh) & mask_valid
    cold_mask = (z < -z_thresh) & mask_valid
    out = np.zeros_like(arr, dtype=int)
    out[hot_mask] = 1
    out[cold_mask] = -1
    return out


# ============================== Plotting ===============================

def imshow_side_by_side(arr: np.ndarray,
                        mask_valid: np.ndarray,
                        gm: np.ndarray,
                        hc_mask: np.ndarray,
                        hot_count: int,
                        cold_count: int) -> None:
    """
    Display two panels:
      • Left: Sobel gradient magnitude (with colorbar; title shows p50/p95/p99)
      • Right: Radiance grayscale with hot (red) / cold (blue) overlay (with grayscale colorbar)
    """
    # Radiance background stretch (2–98%) over valid pixels
    vals = arr[mask_valid]
    if vals.size:
        lo, hi = np.percentile(vals, [2, 98])
    else:
        lo, hi = 0, 1
    normed = (np.clip(arr, lo, hi) - lo) / (hi - lo + 1e-6)
    normed = np.where(mask_valid, normed, np.nan)

    # RGB grayscale base
    base = np.zeros((*arr.shape, 3), dtype=float)
    for c in range(3):
        base[..., c] = normed

    # Overlay hot (red) and cold (blue)
    base[hc_mask == 1] = [1, 0, 0]
    base[hc_mask == -1] = [0, 0, 1]

    # Gradient percentiles for left panel title
    p50, p95, p99 = np.nanpercentile(gm, [50, 95, 99])

    # Plot side by side
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))

    # Left = Sobel gradient (heatmap)
    lo_g, hi_g = 0.0, float(np.nanpercentile(gm, 99))
    im0 = axes[0].imshow(gm, vmin=lo_g, vmax=hi_g, interpolation="nearest")
    axes[0].set_title(f"Sobel gradient (p50={p50:.3f}, p95={p95:.3f}, p99={p99:.3f})")
    axes[0].set_xticks([]); axes[0].set_yticks([])
    cbar0 = fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)
    cbar0.set_label("Gradient magnitude")

    # Right = Radiance with hot/cold overlay
    im1 = axes[1].imshow(base, interpolation="nearest")
    axes[1].set_title(f"Radiance (gray) + Hot=red / Cold=blue\nHot={hot_count}, Cold={cold_count}")
    axes[1].set_xticks([]); axes[1].set_yticks([])
    cbar1 = fig.colorbar(plt.cm.ScalarMappable(cmap="gray"),
                         ax=axes[1], fraction=0.046, pad=0.04)
    cbar1.set_label("Radiance (normalized 2–98%)")

    plt.tight_layout()
    plt.show()


# ============================ QC Table Core ============================

def overall_health_table(geotiff_path: str | Path) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    Compute overall QC metrics and return (table_dataframe, extras_dict).

    Notes
    -----
    • The returned DataFrame has one row and is later printed transposed.
    • 'extras' contains arrays and counts used for visualization.
    """
    geotiff_path = Path(geotiff_path)
    assert geotiff_path.exists(), f"Not found: {geotiff_path}"

    # ---- Read raster band & metadata ----
    with rasterio.open(geotiff_path) as ds:
        arr = ds.read(1)
        nodata_tag = ds.nodata
        h, w = ds.height, ds.width
        crs = ds.crs.to_string() if ds.crs else "None"

    # ---- Masks & hole analysis ----
    mask_valid = build_valid_mask(arr, nodata_tag)
    nodata = ~mask_valid
    total = arr.size
    valid_pixels = int(mask_valid.sum())

    holes_mask, hole_sizes = internal_holes_mask_and_sizes(nodata)
    internal_holes_count = hole_sizes.size
    internal_holes_pixels = int(hole_sizes.sum()) if internal_holes_count else 0
    internal_holes_ratio = float(internal_holes_pixels / total) if total else np.nan

    # ---- Radiance stats (valid only) ----
    vals = arr[mask_valid]
    if vals.size:
        vmin, vmax = float(np.min(vals)), float(np.max(vals))
        vmean, vstd = float(np.mean(vals)), float(np.std(vals))
        p2, p98 = robust_percentiles(vals, (2, 98))
    else:
        vmin = vmax = vmean = vstd = p2 = p98 = np.nan

    # ---- Outlier counts (hot/cold) ----
    if vals.size:
        m, s = np.mean(vals), np.std(vals)
        s = s if s != 0 else 1.0
        z = (arr - m) / s
        hot = int(np.sum((z > 6) & mask_valid))
        cold = int(np.sum((z < -6) & mask_valid))
    else:
        hot = cold = 0

    # ---- Drop-line candidates (row/col mean |z| > 5) ----
    with np.errstate(invalid="ignore"):
        row_means = np.where(np.any(mask_valid, axis=1),
                             np.nanmean(np.where(mask_valid, arr, np.nan), axis=1), np.nan)
        col_means = np.where(np.any(mask_valid, axis=0),
                             np.nanmean(np.where(mask_valid, arr, np.nan), axis=0), np.nan)
    row_z = zscore_1d(row_means)
    col_z = zscore_1d(col_means)
    row_drops = int(np.nansum(np.abs(row_z) > 5))
    col_drops = int(np.nansum(np.abs(col_z) > 5))

    # ---- Gradient distribution percentiles ----
    gstats = gradient_percentiles(arr, mask_valid)

    # ---- Assemble table (single-row DataFrame) ----
    table = {
        "CRS": crs,
        "Image size": f"{h} × {w}",
        "Valid pixels (count)": valid_pixels,
        "NoData holes (count)": internal_holes_count,
        "NoData hole pixels (total)": internal_holes_pixels,
        "NoData hole ratio": round(internal_holes_ratio, 6) if np.isfinite(internal_holes_ratio) else np.nan,
        "Radiance min": vmin,
        "Radiance mean": vmean,
        "Radiance max": vmax,
        "Std. dev. (valid)": vstd,
        "p2 (valid)": p2,
        "p98 (valid)": p98,
        "Hot pixels (z>6)": hot,
        "Cold pixels (z<-6)": cold,
        "Drop-line rows (|z|>5)": row_drops,
        "Drop-line cols (|z|>5)": col_drops,
        "Gradient p50": gstats["grad_p50"],
        "Gradient p95": gstats["grad_p95"],
        "Gradient p99": gstats["grad_p99"],
    }

    def _fmt(v):
        if isinstance(v, (int, np.integer)):
            return int(v)
        if isinstance(v, float) and np.isfinite(v):
            return float(np.round(v, 6))
        return v

    table = {k: _fmt(v) for k, v in table.items()}
    df = pd.DataFrame([table])
    extras = dict(arr=arr, mask_valid=mask_valid, hot=hot, cold=cold)
    return df, extras


# =========================== Descriptions ==============================

METRIC_DESCRIPTIONS = {
    "CRS": "Coordinate Reference System of the raster (e.g., EPSG:4326).",
    "Image size": "Raster height × width (pixels).",
    "Valid pixels (count)": "Number of pixels considered valid (not NoData/≤-1e30).",
    "NoData holes (count)": "Number of interior NoData components not touching any border.",
    "NoData hole pixels (total)": "Total pixels belonging to all interior NoData holes.",
    "NoData hole ratio": "Interior NoData pixels divided by total image pixels.",
    "Radiance min": "Minimum radiance over valid pixels.",
    "Radiance mean": "Mean radiance over valid pixels.",
    "Radiance max": "Maximum radiance over valid pixels.",
    "Std. dev. (valid)": "Standard deviation of radiance over valid pixels.",
    "p2 (valid)": "2nd percentile of radiance over valid pixels (robust low).",
    "p98 (valid)": "98th percentile of radiance over valid pixels (robust high).",
    "Hot pixels (z>6)": "Pixels >6σ brighter than mean; often sensor outliers (red overlay).",
    "Cold pixels (z<-6)": "Pixels <−6σ darker than mean; often sensor outliers (blue overlay).",
    "Drop-line rows (|z|>5)": "Rows whose mean radiance has |z| > 5 (banding candidates).",
    "Drop-line cols (|z|>5)": "Columns whose mean radiance has |z| > 5 (banding candidates).",
    "Gradient p50": "Typical scene smoothness; low in smooth scenes (ocean ≈0.05–0.2), higher in rugged/urban.",
    "Gradient p95": "Strength of common strong edges (coastlines/cloud edges); unusually high in smooth scenes ⇒ noise/striping.",
    "Gradient p99": "Strongest 1% edges/artifacts (sharp land–sea/cloud fronts); very high ⇒ defects.",
}


def describe_metric(metric_name: str) -> str:
    """Return human-readable description for a metric name."""
    return METRIC_DESCRIPTIONS.get(metric_name, "")


def print_transposed(df: pd.DataFrame) -> None:
    """
    Pretty-print QC metrics as a three-column table: Metric | Value | Description.
    (Keeps original functionality of printing to stdout.)
    """
    assert df.shape[0] == 1
    metrics = list(df.columns)
    values = [df.iloc[0, i] for i in range(len(metrics))]
    rows = [{"Metric": m, "Value": v, "Description": describe_metric(m)}
            for m, v in zip(metrics, values)]
    tdf = pd.DataFrame(rows, columns=["Metric", "Value", "Description"])
    with pd.option_context("display.max_rows", None, "display.width", 170):
        print(tdf.to_string(index=False))


# ================================ CLI =================================

if __name__ == "__main__":
    # Example inputs (toggle as needed)
    INPUT = Path(r"path\to\F002_L1__IR__L2L1M0__2025-08-12T212259.010953Z_2025-08-13T111644.731945Z_e81989f5_MWIR.tif")

    # ---- Compute QC table & print (transposed) ----
    df, extras = overall_health_table(INPUT)
    print_transposed(df)

    # ---- Build layers & visualize side-by-side ----
    gm = compute_sobel_magnitude(extras["arr"], extras["mask_valid"])
    hc_mask = compute_hot_cold_mask(extras["arr"], extras["mask_valid"], z_thresh=6.0)
    imshow_side_by_side(
        extras["arr"], extras["mask_valid"], gm, hc_mask,
        hot_count=extras["hot"], cold_count=extras["cold"]
    )


In [ ]:
# ================ forest2_plus_reference_visualize.py ======================
# FOREST-2 MWIR Radiance + Brightness Temperature (Manual Time Zone)
# VIIRS/MODIS Comparison + Points Overlay
#
# Purpose
#   • Reproject FOREST-2 MWIR radiance to EPSG:4326 and compute BT (λ_eff ≈ 3.8 µm)
#   • Load VIIRS/MODIS Radiance/BT (already in GeoTIFF), crop all rasters by AOI
#   • Compute AOI-based NoData shares
#   • Parse acquisition timestamps (FOREST-2 ISO; VIIRS/MODIS AYYYYDDD.HHMM)
#     and convert to a user-defined local timezone for figure stamps
#   • Overlay detection points (GeoJSON) on map panels (black fill, white edge)
#   • Produce paired maps with a single shared colorbar
#   • Produce overlaid histograms (log-density) for Radiance & BT
#
# Requirements
#   pip install rasterio geopandas shapely matplotlib pandas numpy tzdata
#   (Shapely 1.x or 2.x supported)
# ======================================================================

# ============================== Imports ================================
import re
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
from rasterio.features import geometry_mask
from rasterio.transform import array_bounds
from affine import Affine

import geopandas as gpd
from shapely.ops import transform as shp_transform
from shapely.geometry import Point
import shapely


# ============================ User Settings ============================
# --- Input files (adjust as needed) ---
# FOREST-2 L1 MWIR radiance (source CRS; will be reprojected to EPSG:4326)
INPUT_FILE = r"path\to\F002_L1__IR__L2L1M0__2025-01-10T215412.018348Z_2025-04-10T154832.806087Z_97706189_MWIR.tif"

# Reference satellite (example: MODIS Band 21 Radiance / Band 31 BT in GeoTIFF)
REF_SAT_RAD_FILE = r"path\to\MYD021KM.A2025010.2120.061.2025014191057_B21_Rad.tif"
REF_SAT_BT_FILE  = r"path\to\MYD021KM.A2025010.2120.061.2025014191057_B31_BT_K.tif"

# Detection points and AOI boundary
POINTS_GEOJSON = r"path\to\F002_L1__IR__L2L1M0__2025-01-10T215412.018348Z_2025-04-10T154832.806087Z_97706189_model_detections.geojson"
BOUNDARY_SHP   = r"path\to\F002_L1__IR__L2L1M0__2025-01-10T215412.018348Z_2025-04-10T154832.806087Z_97706189_MWIR_Boundary.shp"

# --- Timezone controls ---
USE_LOCAL_TZ_F2   = True
USE_LOCAL_TZ_REF  = True
TZ_NAME_LOCAL     = "America/Los_Angeles"  # e.g., "America/Chicago"

# --- Processing & visualization parameters ---
TARGET_CRS        = CRS.from_epsg(4326)   # output CRS for FOREST-2 reprojection
PERC_STRETCH      = (2, 98)               # robust percentile stretch for quicklooks
SAVE_COMPRESSION  = "lzw"                 # GeoTIFF compression for outputs (None to disable)
WRITE_TIME_TAGS   = True                  # write acquisition time tags to outputs
ALL_TOUCHED       = False                 # rasterization behavior for AOI masks
ASSUME_POINTS_CRS = CRS.from_epsg(4326)   # assumed CRS if GeoJSON lacks CRS
DRAW_AOI_OUTLINE  = False                 # set True to draw AOI outline on maps
ADDITIONAL_NODATA_VALUES = (-3.40282e38, 65535, 65555)  # harden NoData detection


# ============================ Helper Utils ============================
def robust_percentiles(arr: np.ndarray, p_lo=2, p_hi=98):
    """Return robust low/high percentiles ignoring non-finite values."""
    f = np.isfinite(arr)
    if f.any():
        lo, hi = np.percentile(arr[f], [p_lo, p_hi])
        return float(lo), float(hi)
    return np.nan, np.nan


def pooled_percentiles(a: np.ndarray, b: np.ndarray, p_lo=2, p_hi=98):
    """Pooled robust percentiles across two arrays."""
    a_lo, a_hi = robust_percentiles(a, p_lo, p_hi)
    b_lo, b_hi = robust_percentiles(b, p_lo, p_hi)
    return np.nanmin([a_lo, b_lo]), np.nanmax([a_hi, b_hi])


def _parse_start_iso_from_name(name: str):
    """FOREST-2: first ISO-8601 timestamp in filename → aware UTC datetime, if present."""
    m = re.search(
        r"(?P<date>\d{4}-\d{2}-\d{2})T(?P<hh>\d{2})(?P<mm>\d{2})(?P<ss>\d{2})(?:\.(?P<frac>\d+))?Z",
        name
    )
    if not m:
        return None
    parts = m.groupdict()
    micro = int((parts.get("frac") or "0")[:6].ljust(6, "0"))
    return datetime(
        *map(int, parts["date"].split("-")),
        int(parts["hh"]), int(parts["mm"]), int(parts["ss"]),
        microsecond=micro, tzinfo=timezone.utc
    )


def extract_f2_datetime_utc(path: Path):
    """FOREST-2 UTC acquisition time from filename or GeoTIFF tags (best-effort)."""
    dt = _parse_start_iso_from_name(path.name)
    if dt:
        return dt
    try:
        with rasterio.open(path) as src:
            tags = src.tags()
        for key in ("ACQ_DATETIME", "ACQUISITION_DATETIME", "ACQ_TIME", "DATETIME", "TIFFTAG_DATETIME"):
            val = tags.get(key)
            if not val:
                continue
            if key == "TIFFTAG_DATETIME":
                # e.g., 2025:01:10 21:54:12
                return datetime.strptime(val, "%Y:%m:%d %H:%M:%S").replace(tzinfo=timezone.utc)
            # ISO with potential trailing 'Z'
            return datetime.fromisoformat(val.replace("Z", "+00:00")).astimezone(timezone.utc)
    except Exception:
        return None


def parse_ref_utc_from_name(name: str):
    """VIIRS/MODIS: parse UTC from 'AYYYYDDD.HHMM' (e.g., A2025224.1954); return (dt_utc, code)."""
    m = re.search(r"A(?P<y>\d{4})(?P<doy>\d{3})\.(?P<h>\d{2})(?P<m>\d{2})", name)
    if not m:
        return None, None
    y, doy, hh, mm = int(m["y"]), int(m["doy"]), int(m["h"]), int(m["m"])
    code = f"{y}{doy:03d}.{hh:02d}{mm:02d}"
    dt_utc = datetime(y, 1, 1, tzinfo=timezone.utc) + timedelta(days=doy - 1, hours=hh, minutes=mm)
    return dt_utc, code


def to_local_naive(dt_utc, use_local: bool, tz_name: str):
    """UTC → naive local datetime (with a label) or naive UTC if disabled/invalid."""
    if dt_utc is None:
        return None, None
    if not use_local or not tz_name:
        return dt_utc.replace(tzinfo=None), "UTC"
    try:
        tz = ZoneInfo(tz_name)
        return dt_utc.astimezone(tz).replace(tzinfo=None), tz.key
    except Exception:
        return dt_utc.replace(tzinfo=None), "UTC"


def crop_array_and_transform(arr: np.ndarray, mask: np.ndarray | None, transform: Affine):
    """
    Crop array/mask to the minimal bounding rectangle of True values in mask.
    Returns: arr_crop, mask_crop, transform_crop, extent=(left, right, bottom, top)
    """
    if mask is None or not np.any(mask):
        h, w = arr.shape
        left, bottom, right, top = array_bounds(h, w, transform)
        return arr, mask, transform, (left, right, bottom, top)

    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    r0, r1 = np.where(rows)[0][[0, -1]]
    c0, c1 = np.where(cols)[0][[0, -1]]

    arrc = arr[r0:r1 + 1, c0:c1 + 1]
    maskc = mask[r0:r1 + 1, c0:c1 + 1]
    trc = transform * Affine.translation(c0, r0)

    h, w = arrc.shape
    left, bottom, right, top = array_bounds(h, w, trc)
    return arrc, maskc, trc, (left, right, bottom, top)


def pixel_size_from_transform(tr: Affine) -> float:
    """Return a representative pixel size from an affine transform."""
    return max(abs(tr.a), abs(tr.e))


def union_geoms(geoms):
    """
    Shapely-compatible geometry union:
      • Prefer shapely.ops.union_all (Shapely ≥ 2.0),
      • Fall back to shapely.ops.unary_union (Shapely 1.x).
    """
    try:
        from shapely.ops import union_all as _union_all
        return _union_all(list(geoms))
    except Exception:
        from shapely.ops import unary_union as _unary_union
        return _unary_union(list(geoms))


def _to_2d(x, y, z=None):
    """Helper for dropping any Z component with shapely.ops.transform."""
    return (x, y)


def force_points_2d_centroids(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Ensure point-like features:
      • Explode multipart geometries,
      • Make valid (fallback buffer(0) if needed),
      • Force 2D,
      • Convert non-point geometries to centroids.
    """
    if gdf.empty:
        return gdf
    try:
        gdf = gdf.explode(index_parts=False, ignore_index=True)
    except TypeError:  # older geopandas
        gdf = gdf.explode(ignore_index=True)

    def _fix2d(g):
        if g is None or g.is_empty:
            return g
        try:
            g = shapely.make_valid(g) if hasattr(shapely, "make_valid") else g.buffer(0)
        except Exception:
            pass
        try:
            g = shp_transform(_to_2d, g)
        except Exception:
            pass
        return g

    gdf = gdf.copy()
    gdf["geometry"] = gdf["geometry"].apply(_fix2d)
    gdf["geometry"] = gdf["geometry"].apply(lambda g: g if isinstance(g, Point) else g.centroid)
    return gdf[~gdf.geometry.is_empty & gdf.geometry.notnull()].copy()


def select_points_for_panel(points_path, assume_crs_if_missing, target_crs, aoi_gdf,
                            panel_transform: Affine, panel_extent: tuple, buffer_pixels=1.0):
    """
    Select detection points for a given panel:
      • Reproject points and AOI to panel CRS,
      • Buffer the AOI by ~N pixels (map units),
      • Keep points intersecting buffered AOI.
    """
    pts_raw = gpd.read_file(points_path)
    if pts_raw.crs is None:
        pts_raw = pts_raw.set_crs(assume_crs_if_missing)
    pts_raw = force_points_2d_centroids(pts_raw)

    pts = pts_raw.to_crs(target_crs)
    aoi = aoi_gdf.to_crs(target_crs)
    px = pixel_size_from_transform(panel_transform)
    buf = float(buffer_pixels) * (px if np.isfinite(px) else 0.0)

    geom = union_geoms(aoi.geometry.values)
    try:
        geom_b = geom.buffer(buf) if buf > 0 else geom
    except Exception:
        geom_b = geom

    kept = pts[pts.intersects(geom_b)]
    if len(kept) == 0:
        # Keep empty result predictable but non-crashing (optional behavior)
        kept = pts.copy()
        kept["__force__"] = True
    return kept


# ========================= Visualization Utils ========================
def show_pair_with_right_cbar(arr_left, title_left, stamp_left, extent_left, pts_left, aoi_left,
                              arr_right, title_right, stamp_right, extent_right, pts_right, aoi_right,
                              vmin, vmax, cmap, cbar_label):
    """
    Create two side-by-side map panels with a shared colorbar on the far right.
    Titles/timestamps are placed above to avoid overlaying imagery.
    """
    # Aspect ratios (map units)
    lw = extent_left[1] - extent_left[0]
    lh = extent_left[3] - extent_left[2]
    rw = extent_right[1] - extent_right[0]
    rh = extent_right[3] - extent_right[2]
    ar_left = (lh / lw) if lw > 0 else 1.0
    ar_right = (rh / rw) if rw > 0 else 1.0

    # Balanced figure size from aspect ratios
    base_h = 4.6
    avg_inv = 0.5 * ((1.0 / max(ar_left, 1e-6)) + (1.0 / max(ar_right, 1e-6)))
    fig_w = base_h * avg_inv * 2.0 + 0.9
    fig_h = base_h + 0.9
    fig = plt.figure(figsize=(fig_w, fig_h))

    # Grid: [title row; image row] x [left image, right image, spacer, colorbar]
    gs = fig.add_gridspec(
        2, 4,
        height_ratios=[0.09, 0.91],
        width_ratios=[1.0, 1.0, 0.004, 0.033],
        wspace=0.01, hspace=0.01
    )

    # Title/timestamp areas
    ax_stamp_l = fig.add_subplot(gs[0, 0])
    ax_stamp_r = fig.add_subplot(gs[0, 1])
    for ax in (ax_stamp_l, ax_stamp_r):
        ax.axis("off")
    ax_stamp_l.text(0.0, 0.55, f"{title_left}\n{stamp_left}", ha="left", va="center", fontsize=10)
    ax_stamp_r.text(0.0, 0.55, f"{title_right}\n{stamp_right}", ha="left", va="center", fontsize=10)

    # Image axes and shared colorbar axis
    ax1 = fig.add_subplot(gs[1, 0])
    ax2 = fig.add_subplot(gs[1, 1])
    cax = fig.add_subplot(gs[:, 3])  # shared colorbar at far right

    for ax in (ax1, ax2):
        ax.set_xticks([]); ax.set_yticks([])
        for s in ax.spines.values():
            s.set_visible(False)
        ax.set_anchor('W')
    ax1.set_box_aspect(ar_left)
    ax2.set_box_aspect(ar_right)

    # Draw rasters (extent = left, right, bottom, top)
    im1 = ax1.imshow(arr_left,  vmin=vmin, vmax=vmax, cmap=cmap, extent=extent_left,  origin="upper")
    im2 = ax2.imshow(arr_right, vmin=vmin, vmax=vmax, cmap=cmap, extent=extent_right, origin="upper")

    # Lock extents
    ax1.set_xlim(extent_left[0],  extent_left[1]);  ax1.set_ylim(extent_left[2],  extent_left[3])
    ax2.set_xlim(extent_right[0], extent_right[1]); ax2.set_ylim(extent_right[2], extent_right[3])

    # Optional AOI outline
    if DRAW_AOI_OUTLINE:
        try:
            aoi_left.boundary.plot(ax=ax1,  color="white", linewidth=0.8, alpha=0.9, zorder=5)
            aoi_right.boundary.plot(ax=ax2, color="white", linewidth=0.8, alpha=0.9, zorder=5)
        except Exception:
            pass

    # Points overlay
    if pts_left is not None and not pts_left.empty:
        pts_left.plot(ax=ax1, color="black", edgecolor="white", markersize=70, linewidth=1.0, zorder=20)
    if pts_right is not None and not pts_right.empty:
        pts_right.plot(ax=ax2, color="black", edgecolor="white", markersize=70, linewidth=1.0, zorder=20)

    # Shared colorbar
    cb = fig.colorbar(im1, cax=cax, orientation="vertical")
    cb.set_label(cbar_label)
    cb.ax.yaxis.set_tick_params(pad=2)

    # Tight layout
    fig.subplots_adjust(left=0.018, right=0.988, top=0.96, bottom=0.065, wspace=0.01, hspace=0.01)
    plt.show()


# ============================ I/O Utilities ===========================
def load_single_band(path_str: str):
    """
    Load a single-band GeoTIFF as float64 with NaN NoData.
    Detects embedded NoData plus common artifact values (configurable).
    """
    p = Path(path_str)
    assert p.exists(), f"Not found: {p}"
    with rasterio.open(p) as ds:
        arr = ds.read(1).astype(np.float64)
        nd = ds.nodata

        # Normalize NoData: declared NoData or known fill values → NaN
        mask_declared = np.zeros_like(arr, dtype=bool)
        if nd is not None and np.isfinite(nd):
            mask_declared |= (arr == nd)

        mask_artifacts = np.zeros_like(arr, dtype=bool)
        for val in ADDITIONAL_NODATA_VALUES:
            mask_artifacts |= (arr == val)

        arr = np.where(mask_declared | mask_artifacts, np.nan, arr)

        return arr, ds.crs, ds.transform, nd


# ============================== Main Flow =============================
def main():
    # ---------- Reproject FOREST-2 radiance to TARGET_CRS ----------
    in_path = Path(INPUT_FILE)
    assert in_path.exists(), f"Not found: {in_path}"

    f2_dt_utc = extract_f2_datetime_utc(in_path)  # aware UTC (or None)
    with rasterio.open(in_path) as src:
        assert src.crs, "FOREST-2 input has no CRS."
        out_transform, out_w, out_h = calculate_default_transform(
            src.crs, TARGET_CRS, src.width, src.height, *src.bounds
        )
        rad_f2 = np.full((out_h, out_w), np.nan, dtype=np.float64)
        reproject(
            source=src.read(1).astype(np.float64),
            destination=rad_f2,
            src_transform=src.transform,
            src_crs=src.crs,
            src_nodata=src.nodata,
            dst_transform=out_transform,
            dst_crs=TARGET_CRS,
            dst_nodata=np.nan,                # force NaN NoData in output
            resampling=Resampling.bilinear
        )

    # Optional: save reprojected FOREST-2 radiance
    crs_tag = f"epsg{TARGET_CRS.to_epsg()}" if TARGET_CRS.to_epsg() else "reproj"
    out_rad_path = in_path.with_name(in_path.stem + f"_radiance_{crs_tag}.tif")
    if SAVE_COMPRESSION:
        with rasterio.open(
            out_rad_path, "w",
            driver="GTiff",
            height=rad_f2.shape[0], width=rad_f2.shape[1],
            count=1, dtype=rasterio.float32,
            crs=TARGET_CRS, transform=out_transform,
            nodata=np.float32(np.nan), compress=SAVE_COMPRESSION
        ) as dst:
            dst.write(rad_f2.astype(np.float32), 1)
            if WRITE_TIME_TAGS and isinstance(f2_dt_utc, datetime):
                dst.update_tags(ACQ_DATETIME_UTC=f2_dt_utc.strftime("%Y-%m-%d %H:%M:%S"))

    # Local time stamp for figure annotations
    f2_dt_local, f2_tz_label = to_local_naive(f2_dt_utc, USE_LOCAL_TZ_F2, TZ_NAME_LOCAL)
    f2_stamp = (
        f2_dt_local.strftime("%Y-%m-%d %H:%M") + (f" {f2_tz_label}" if f2_tz_label else "")
    ) if isinstance(f2_dt_local, datetime) else "Time: unknown"

    # ---------- AOI masks & crops ----------
    gdf_raw = gpd.read_file(BOUNDARY_SHP)
    assert gdf_raw.crs, "Boundary shapefile has no CRS."

    # FOREST-2 boundary/mask/crop
    gdf_f2 = gdf_raw.to_crs(TARGET_CRS)
    geom_f2 = union_geoms(gdf_f2.geometry.values)
    mask_f2 = geometry_mask([geom_f2], out_shape=rad_f2.shape, transform=out_transform,
                            invert=True, all_touched=ALL_TOUCHED)
    rad_f2_crop, _, transform_f2_crop, extent_f2 = crop_array_and_transform(rad_f2, mask_f2, out_transform)

    # ---------- Brightness Temperature (Planck inversion at λ_eff ≈ 3.8 µm) ----------
    # CODATA constants (exact in SI):
    h = 6.62607015e-34      # Planck constant [J·s]
    c = 2.99792458e8        # speed of light [m/s]
    k = 1.380649e-23        # Boltzmann constant [J/K]
    lam_eff_um = 3.8
    lam = lam_eff_um * 1e-6

    # K1, K2 for monochromatic radiance at λ (per-µm units)
    K1 = (2 * h * c**2) / (lam**5) * 1e-6
    K2 = (h * c) / (k * lam)

    # Brightness temperature from radiance: Tb = K2 / ln(1 + K1 / Lλ)
    # Clip radiance to avoid division and log underflow
    bt_f2 = K2 / np.log1p(K1 / np.clip(rad_f2, 1e-6, None))
    bt_f2_crop, _, _, extent_f2_bt = crop_array_and_transform(bt_f2, mask_f2, out_transform)

    # NoData shares (FOREST-2)
    total_in_f2 = int(mask_f2.sum())
    nodata_share_rad_f2 = (np.isnan(rad_f2)[mask_f2].sum() / total_in_f2) if total_in_f2 > 0 else np.nan
    nodata_share_bt_f2  = ((~np.isfinite(bt_f2))[mask_f2].sum() / total_in_f2) if total_in_f2 > 0 else np.nan

    # ---------- Reference satellite (VIIRS/MODIS) load & crop ----------
    ref_rad, v_crs,  v_transform,  _ = load_single_band(REF_SAT_RAD_FILE)
    ref_bt,  v_crs2, v_transform2, _ = load_single_band(REF_SAT_BT_FILE)

    # Reference time (UTC) → local
    v_dt_utc, v_timecode = parse_ref_utc_from_name(Path(REF_SAT_RAD_FILE).name)
    if v_dt_utc is None:
        v_dt_utc, v_timecode = parse_ref_utc_from_name(Path(REF_SAT_BT_FILE).name)
    v_dt_local, v_tz_label = to_local_naive(v_dt_utc, USE_LOCAL_TZ_REF, TZ_NAME_LOCAL)
    v_stamp = (
        v_dt_local.strftime("%Y-%m-%d %H:%M") + (f" {v_tz_label}" if v_tz_label else "")
    ) if isinstance(v_dt_local, datetime) else "Time: unknown"

    # Reference boundary/masks/crops (Radiance)
    gdf_vr = gdf_raw.to_crs(v_crs)
    geom_vr = union_geoms(gdf_vr.geometry.values)
    mask_vr = geometry_mask([geom_vr], out_shape=ref_rad.shape, transform=v_transform,
                            invert=True, all_touched=ALL_TOUCHED)
    ref_rad_crop, _, _, extent_vr = crop_array_and_transform(ref_rad, mask_vr, v_transform)

    # Reference boundary/masks/crops (BT)
    gdf_vb = gdf_raw.to_crs(v_crs2)
    geom_vb = union_geoms(gdf_vb.geometry.values)
    mask_vb = geometry_mask([geom_vb], out_shape=ref_bt.shape, transform=v_transform2,
                            invert=True, all_touched=ALL_TOUCHED)
    ref_bt_crop, _, _, extent_vb = crop_array_and_transform(ref_bt, mask_vb, v_transform2)

    # NoData shares (Reference)
    total_in_vr = int(mask_vr.sum())
    total_in_vb = int(mask_vb.sum())
    nodata_share_rad_ref = (np.isnan(ref_rad)[mask_vr].sum() / total_in_vr) if total_in_vr > 0 else np.nan
    nodata_share_bt_ref  = ((~np.isfinite(ref_bt))[mask_vb].sum() / total_in_vb) if total_in_vb > 0 else np.nan

    # ---------- Points for overlays ----------
    pts_f2_for_rad = select_points_for_panel(
        POINTS_GEOJSON, ASSUME_POINTS_CRS, TARGET_CRS, gdf_raw, transform_f2_crop, extent_f2, buffer_pixels=1.0
    )
    pts_f2_for_bt  = pts_f2_for_rad
    pts_ref_for_rad = select_points_for_panel(
        POINTS_GEOJSON, ASSUME_POINTS_CRS, v_crs,  gdf_raw, v_transform,  extent_vr, buffer_pixels=1.0
    )
    pts_ref_for_bt  = select_points_for_panel(
        POINTS_GEOJSON, ASSUME_POINTS_CRS, v_crs2, gdf_raw, v_transform2, extent_vb, buffer_pixels=1.0
    )

    # ---------- Stretches for display ----------
    def pooled(v1, v2):
        vmin = np.nanmin([
            np.nanpercentile(v1[np.isfinite(v1)], PERC_STRETCH[0]) if np.isfinite(v1).any() else np.nan,
            np.nanpercentile(v2[np.isfinite(v2)], PERC_STRETCH[0]) if np.isfinite(v2).any() else np.nan
        ])
        vmax = np.nanmax([
            np.nanpercentile(v1[np.isfinite(v1)], PERC_STRETCH[1]) if np.isfinite(v1).any() else np.nan,
            np.nanpercentile(v2[np.isfinite(v2)], PERC_STRETCH[1]) if np.isfinite(v2).any() else np.nan
        ])
        return vmin, vmax

    rad_vmin, rad_vmax = pooled(rad_f2_crop, ref_rad_crop)
    bt_vmin,  bt_vmax  = pooled(bt_f2_crop,  ref_bt_crop)

    # ============================ Visuals =============================
    # Radiance paired maps
    show_pair_with_right_cbar(
        rad_f2_crop, "FOREST-2 MWIR Radiance", f2_stamp, extent_f2, pts_f2_for_rad, gdf_f2,
        ref_rad_crop, "Reference MWIR Radiance",  v_stamp, extent_vr, pts_ref_for_rad, gdf_vr,
        rad_vmin, rad_vmax, "plasma", "Radiance (W m$^{-2}$ sr$^{-1}$ µm$^{-1}$)"
    )

    # Radiance histogram (overlaid)
    f2r_f = np.isfinite(rad_f2_crop); vr_f = np.isfinite(ref_rad_crop)
    lo = np.nanmin([np.nanmin(rad_f2_crop[f2r_f]) if f2r_f.any() else np.nan,
                    np.nanmin(ref_rad_crop[vr_f]) if vr_f.any() else np.nan])
    hi = np.nanmax([np.nanmax(rad_f2_crop[f2r_f]) if f2r_f.any() else np.nan,
                    np.nanmax(ref_rad_crop[vr_f]) if vr_f.any() else np.nan])
    bins = np.linspace(lo, hi, 200) if (np.isfinite(lo) and np.isfinite(hi) and hi > lo) else 200

    plt.figure(figsize=(8.4, 5.0))
    if f2r_f.any():
        plt.hist(rad_f2_crop[f2r_f].ravel(), bins=bins, density=True, label="FOREST-2", alpha=0.85)
    if vr_f.any():
        plt.hist(ref_rad_crop[vr_f].ravel(), bins=bins, density=True, label="Reference MWIR", color="red", alpha=0.7)
    plt.yscale("log")
    plt.xlabel("Radiance (W m$^{-2}$ sr$^{-1}$ µm$^{-1}$)")
    plt.ylabel("Density (log)")
    plt.title("Radiance Distribution")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # BT paired maps
    show_pair_with_right_cbar(
        bt_f2_crop, "FOREST-2 Brightness Temperature", f2_stamp, extent_f2_bt, pts_f2_for_bt, gdf_f2,
        ref_bt_crop, "Reference Brightness Temperature",  v_stamp, extent_vb, pts_ref_for_bt, gdf_vb,
        bt_vmin, bt_vmax, "Spectral_r", "Brightness Temperature (K)"
    )

    # BT histogram (overlaid)
    f2b_f = np.isfinite(bt_f2_crop); vb_f = np.isfinite(ref_bt_crop)
    lo = np.nanmin([np.nanmin(bt_f2_crop[f2b_f]) if f2b_f.any() else np.nan,
                    np.nanmin(ref_bt_crop[vb_f]) if vb_f.any() else np.nan])
    hi = np.nanmax([np.nanmax(bt_f2_crop[f2b_f]) if f2b_f.any() else np.nan,
                    np.nanmax(ref_bt_crop[vb_f]) if vb_f.any() else np.nan])
    bins = np.linspace(lo, hi, 200) if (np.isfinite(lo) and np.isfinite(hi) and hi > lo) else 200

    plt.figure(figsize=(8.4, 5.0))
    if f2b_f.any():
        plt.hist(bt_f2_crop[f2b_f].ravel(), bins=bins, density=True, label="FOREST-2", alpha=0.85)
    if vb_f.any():
        plt.hist(ref_bt_crop[vb_f].ravel(), bins=bins, density=True, label="Reference MWIR", color="red", alpha=0.7)
    plt.yscale("log")
    plt.xlabel("Brightness Temperature (K)")
    plt.ylabel("Density (log)")
    plt.title("Brightness Temperature Distribution")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # ---------- Optional: save FOREST-2 BT ----------
    out_bt_path = in_path.with_name(in_path.stem + f"_BT_K_{crs_tag}.tif")
    if SAVE_COMPRESSION:
        with rasterio.open(
            out_bt_path, "w",
            driver="GTiff",
            height=bt_f2.shape[0], width=bt_f2.shape[1],
            count=1, dtype=rasterio.float32,
            crs=TARGET_CRS, transform=out_transform,
            nodata=np.float32(np.nan), compress=SAVE_COMPRESSION
        ) as dst:
            dst.write(bt_f2.astype(np.float32), 1)
            if WRITE_TIME_TAGS and isinstance(f2_dt_utc, datetime):
                dst.update_tags(
                    ACQ_DATETIME_UTC=f2_dt_utc.strftime("%Y-%m-%d %H:%M:%S"),
                    ACQ_TZ=(f2_tz_label or "UTC"),
                    ACQ_DATETIME_LOCAL=(f2_dt_local.strftime("%Y-%m-%d %H:%M:%S")
                                        if isinstance(f2_dt_local, datetime) else "unknown")
                )

if __name__ == "__main__":
    main()


In [ ]:
# ================ air_temperature.py ======================
"""
Daily near-surface air temperature (Ta_C) and shortwave radiation (Rs)
over an AOI from NOAA CFSv2 FOR6H, aggregated to daily means via a
time-warp helper, and returned as a pandas DataFrame `df_daily`.

Outputs
-------
• `df_daily`  (pandas.DataFrame): columns = ['date', 'Ta_C', 'Rs']
   - 'date' is a pandas datetime64[ns] (no timezone) sorted ascending
   - Ta_C in °C, Rs in W m^-2 (6-hour average band daily-mean)

Notes
-----
• Requires an authenticated Earth Engine account.

Dependencies
----
Install earthengine-api geemap pandas
"""

# ============================== Imports ===============================
import ee
import geemap
import pandas as pd
from geemap import shp_to_ee  # noqa: F401  (kept for reference in docstring)

# ======================== Earth Engine Init ===========================
# Authenticate once if needed, then initialize. This keeps behavior intact.
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# =========================== Configuration ============================
# AOI shapefile (single or multi-part polygon)
shapefile_path = r'path\to\F002_L1__IR__L2L1M0__2025-01-10T215412.018348Z_2025-04-10T154832.806087Z_97706189_MWIR_Boundary.shp'

# Analysis window (inclusive start, exclusive end in EE filterDate semantics)
startDate = ee.Date.fromYMD(2025, 1, 4)
endDate   = ee.Date.fromYMD(2025, 1, 13)

# ============================== AOI ================================
# Convert shapefile to an EE geometry and ensure we use the geometry only (no properties)
AOI = geemap.shp_to_ee(shapefile_path).geometry()

# =========================== Time Helper ============================
def time_warp(collection: ee.ImageCollection,
              start: ee.Date,
              count: ee.Number,
              interval: int,
              units: str) -> ee.ImageCollection:
    """
    Build a daily (or generic) sequence of mean images with aligned timestamps.

    Parameters
    ----------
    collection : ee.ImageCollection
        Source images to aggregate (already filtered by date/bounds).
    start : ee.Date
        Start date of the first interval.
    count : ee.Number
        Number of intervals to produce.
    interval : int
        Size of each interval (e.g., 1 for daily).
    units : str
        Units for 'interval' (e.g., 'day').

    Returns
    -------
    ee.ImageCollection
        Collection where each image is the mean over one interval with:
        - 'system:time_start' set to the interval start
        - 'date_daily', 'system:id', 'system:index' set to YYYY-MM-DD
    """
    sequence = ee.List.sequence(0, ee.Number(count).subtract(1))
    origin_date = ee.Date(start)

    def _each(i):
        i = ee.Number(i)
        start_date = origin_date.advance(i.multiply(interval), units)
        end_date   = origin_date.advance(i.add(1).multiply(interval), units)
        img = (collection
               .filterDate(start_date, end_date)
               .mean()
               .set('system:time_start', start_date.millis())
               .set('date_daily', start_date.format('YYYY-MM-dd'))
               .set('system:id', start_date.format('YYYY-MM-dd'))
               .set('system:index', start_date.format('YYYY-MM-dd')))
        return img

    return ee.ImageCollection(sequence.map(_each))

# ======================= Source Dataset (CFSv2) ======================
# NOAA/CFSV2/FOR6H: 6-hourly forecast fields
NCEP_raw = (ee.ImageCollection("NOAA/CFSV2/FOR6H")
            .filterDate(startDate, endDate)
            .filterBounds(AOI)
            .select([
                "Temperature_height_above_ground",
                "Downward_Short-Wave_Radiation_Flux_surface_6_Hour_Average"
            ]))

# ================== Daily Aggregation (Ta_C, Rs) =====================
# Build daily bins using time_warp, convert Kelvin to °C, clip to AOI, keep metadata.
NCEP_daily = (time_warp(
                    NCEP_raw,
                    start=startDate,
                    count=endDate.difference(startDate, 'day'),
                    interval=1,
                    units='day')
              .map(lambda image:
                   (image
                    .select("Downward_Short-Wave_Radiation_Flux_surface_6_Hour_Average")
                    .rename("Rs")
                    .addBands(
                        image.select("Temperature_height_above_ground")
                             .subtract(273.15)  # K → °C
                             .rename("Ta_C")
                    )
                    .clip(AOI)
                    .copyProperties(image, image.propertyNames())
                    .set("date_daily", image.date().format("YYYY-MM-dd"))
                    .set("system:id", image.date().format("YYYY-MM-dd H:mm"))
                    ))
              )

# Bands to export (kept identical to original behavior)
bands_weather_daily = ["Ta_C", "Rs"]

# ======================= Region Reduce to AOI ========================
def _reduce_daily_means(image: ee.Image) -> ee.Feature:
    """
    Reduce selected bands over AOI to daily means.

    Returns
    -------
    ee.Feature
        Feature with properties {'date', 'Ta_C', 'Rs'}
    """
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=AOI,
        scale=30,          # kept as in original script to preserve behavior
        bestEffort=True,
        maxPixels=1e13
    )
    # Attach the date string for tabular export
    return ee.Feature(None, stats.set('date', image.date().format('YYYY-MM-dd')))

features_daily = NCEP_daily.select(bands_weather_daily).map(_reduce_daily_means)

# ========================= Client-Side Table ==========================
# Pull results to client and assemble pandas DataFrame (no prints)
_feature_collection = features_daily.getInfo()  # triggers server → client transfer

_records = []
for feat in _feature_collection['features']:
    props = feat['properties']
    row = [props.get('date')] + [props.get(k, None) for k in bands_weather_daily]
    _records.append(row)

df_daily = pd.DataFrame(_records, columns=['date'] + bands_weather_daily)
df_daily['date'] = pd.to_datetime(df_daily['date'], errors='coerce')
df_daily = df_daily.sort_values('date').reset_index(drop=True)

# Final air temperature DataFrame
df_daily


In [ ]:
# ====================== viirs_timeseries.py ======================
"""
Per-polygon time series of VIIRS I04 Radiance Δ(max − P25)/Δ(max + P25), with DAY & NIGHT lines,
and daily near-surface air temperature (Ta_C) bars from an external DataFrame `df_daily`, generated in air_temperature.py.

Metric per polygon & timestamp
------------------------------
  delta = max(R) − P25(R)        [R in W m^-2 sr^-1 µm^-1]
  R     = pixel radiances within the polygon (NaN-aware)
  P25   = 25th percentile via np.nanpercentile(..., 25)

IMPORTANT: This implementation returns a normalized value:
  delta_norm = (max − P25) / (max + P25)   ∈ [0, 1]
This preserves your current behavior without altering functionality.

Day/Night split
---------------
For each polygon centroid and timestamp, compute Solar Zenith Angle (SZA):
  • Day   if SZA < SZA_DAY_THRESHOLD_DEG (default 88°)
  • Night otherwise

I/O layout
----------
Inputs searched in BASEDIR/YYYY-MM-DD/bt/:  *_I04_Rad.tif
Outputs per polygon:
  • CSV (UTC): datetime_utc, period{day|night}, delta_I04_Wm2_sr_um
  • Inline plot (local time): Day=red solid, Night=blue dashed; points labeled HH:MM
  • Optional background bars (secondary y-axis): daily Ta_C from `df_daily` (date, Ta_C)

Dependencies
------------
  Install numpy pandas rasterio fiona shapely pyproj matplotlib tzdata
"""

from __future__ import annotations

# ============================== Imports ===============================
import re
import csv
import math
from pathlib import Path
from datetime import date, datetime, timedelta, timezone
from typing import List, Tuple, Dict, Any

import numpy as np
import pandas as pd
import rasterio
from rasterio.features import geometry_mask
from shapely.geometry import shape, mapping
from shapely.ops import transform as shp_transform
import fiona
from pyproj import CRS as PJCRS, Transformer

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from zoneinfo import ZoneInfo  # Python 3.9+


# =========================== Configuration ============================

# Base directory containing day folders like YYYY-MM-DD/bt/*.tif
BASEDIR: Path    = Path(r"path\to\base\directory")
START_DATE: date = date(2025, 8, 9)
END_DATE:   date = date(2025, 8, 12)

# Local time zone for plotting (lines and bar labels)
LOCAL_TZNAME: str = "America/Chicago" # e.g., "America/Los_Angeles"
LOCAL_TZ: ZoneInfo = ZoneInfo(LOCAL_TZNAME)

# Day/night threshold (Day if SZA < threshold)
SZA_DAY_THRESHOLD_DEG: float = 88.0

# Polygons shapefile (must contain an integer 'id' or 'ID' attribute)
POLY_SHP: Path = Path(
    r"path\to\F002_L1__IR__L2L1M0__2025-08-12T212259.010953Z_2025-08-13T111644.731945Z_e81989f5_model_detections_Polygon.shp"
)

# Radiance file pattern (inside per-day "bt" folder)
PAT_I04: str = "*_I04_Rad.tif"

# Output directories for per-polygon CSVs
OUTDIR: Path  = BASEDIR / "timeseries"
OUT_CSV: Path = OUTDIR / "csv"
OUT_CSV.mkdir(parents=True, exist_ok=True)


# =========================== Date Utilities ===========================

def date_iter(d0: date, d1: date):
    """Inclusive day iterator from d0 to d1."""
    cur = d0
    while cur <= d1:
        yield cur
        cur += timedelta(days=1)


def timestamp_key_from_name(name: str) -> str | None:
    """Extract '.Ayyyyddd.HHMM.' key from a filename (VIIRS convention)."""
    m = re.search(r"\.(A\d{7}\.\d{4})\.", name)
    return m.group(1) if m else None


def dt_from_key(key: str) -> datetime | None:
    """'.Ayyyyddd.HHMM' → aware UTC datetime."""
    m = re.match(r"A(\d{4})(\d{3})\.(\d{2})(\d{2})$", key)
    if not m:
        return None
    year, doy, hh, mm = map(int, m.groups())
    return datetime(year, 1, 1, tzinfo=timezone.utc) + timedelta(days=doy - 1, hours=hh, minutes=mm)


def gather_i04_index() -> List[Dict[str, Any]]:
    """
    Collect timestamps and file paths for I04 Radiance granules
    across BASEDIR/YYYY-MM-DD/bt/ folders that match PAT_I04.
    Returns a list sorted by UTC datetime.
    """
    recs: Dict[str, Dict[str, Any]] = {}
    for day in date_iter(START_DATE, END_DATE):
        btdir = BASEDIR / day.strftime("%Y-%m-%d") / "bt"
        if not btdir.exists():
            continue
        for p in btdir.glob(PAT_I04):
            key = timestamp_key_from_name(p.name)
            if not key:
                continue
            dt = dt_from_key(key)
            if not dt:
                continue
            recs[key] = {"dt": dt, "i04": p}
    # Sort by datetime (ascending)
    return [recs[k] for k in sorted(recs.keys(), key=lambda kk: recs[kk]["dt"])]


# =========================== Polygon Handling =========================

def read_polygons_wgs84(shp_path: Path) -> List[Tuple[int, Any, float, float]]:
    """
    Read polygon features and return them in EPSG:4326.

    Returns
    -------
    list of tuples
        (poly_id: int, geometry: shapely in EPSG:4326, centroid_lat, centroid_lon)
    """
    if not shp_path.exists():
        raise FileNotFoundError(f"Polygon shapefile not found: {shp_path}")

    polys: List[Tuple[int, Any, float, float]] = []
    with fiona.open(shp_path, "r") as src:
        src_crs = src.crs
        if not src_crs:
            raise ValueError("Input shapefile has no CRS.")
        src_crs_obj = PJCRS.from_user_input(src_crs)
        dst_crs_obj = PJCRS.from_epsg(4326)

        if src_crs_obj != dst_crs_obj:
            transformer = Transformer.from_crs(src_crs_obj, dst_crs_obj, always_xy=True)
            reproj = lambda x, y, z=None: transformer.transform(x, y)
        else:
            reproj = None

        for feat in src:
            props = feat.get("properties", {})
            poly_id = int(props.get("id", props.get("ID", len(polys) + 1)))
            geom = shape(feat["geometry"]).buffer(0)  # make valid
            if reproj:
                geom = shp_transform(reproj, geom)
            c = geom.centroid
            polys.append((poly_id, geom, float(c.y), float(c.x)))  # (id, geom, lat, lon)

    if not polys:
        raise ValueError("No polygons found in shapefile.")
    return polys


# ========================== Solar Geometry ============================

def solar_zenith_deg(lat_deg: float, lon_deg: float, dt_utc: datetime) -> float:
    """
    Approximate Solar Zenith Angle (SZA) in degrees. Sufficient for day/night split.
    """
    y, m, d = dt_utc.year, dt_utc.month, dt_utc.day
    hh, mm = dt_utc.hour, dt_utc.minute
    ss = dt_utc.second + dt_utc.microsecond / 1e6

    # Julian Day (Meeus-like)
    a = math.floor((14 - m) / 12)
    y2 = y + 4800 - a
    m2 = m + 12 * a - 3
    jd = d + math.floor((153 * m2 + 2) / 5) + 365 * y2 + math.floor(y2 / 4) \
         - math.floor(y2 / 100) + math.floor(y2 / 400) - 32045
    frac_day = (hh + mm / 60 + ss / 3600) / 24.0
    JD = jd + frac_day
    T = (JD - 2451545.0) / 36525.0

    # Solar coordinates
    L0 = (280.46646 + 36000.76983 * T + 0.0003032 * T * T) % 360
    M = math.radians((357.52911 + 35999.05029 * T - 0.0001537 * T * T) % 360)
    e = 0.016708634 - 0.000042037 * T - 0.0000001267 * T * T
    C = (1.914602 - 0.004817 * T - 0.000014 * T * T) * math.sin(M) + \
        (0.019993 - 0.000101 * T) * math.sin(2 * M) + 0.000289 * math.sin(3 * M)
    lam_true = math.radians((L0 + C) % 360)
    omega = math.radians(125.04 - 1934.136 * T)
    lam_app = lam_true - math.radians(0.00569) - math.radians(0.00478) * math.sin(omega)

    eps0 = 23 + (26 + (21.448 - T * (46.815 + T * (0.00059 - 0.001813 * T))) / 60) / 60
    eps = math.radians(eps0 + 0.00256 * math.cos(omega))

    # Declination
    sin_delta = math.sin(eps) * math.sin(lam_app)
    delta = math.asin(sin_delta)

    # Equation of Time (minutes)
    y_eps = math.tan(eps / 2) ** 2
    L0r = math.radians(L0)
    E = 4 * math.degrees(
        y_eps * math.sin(2 * L0r) - 2 * e * math.sin(M) + 4 * e * y_eps * math.sin(M) * math.cos(2 * L0r)
        - 0.5 * y_eps * y_eps * math.sin(4 * L0r) - 1.25 * e * e * math.sin(2 * M)
    )

    # True Solar Time (minutes)
    utc_min = hh * 60 + mm + ss / 60
    TST = (utc_min + E + 4 * lon_deg) % 1440
    H = math.radians(TST / 4 - 180)  # hour angle

    # Zenith angle
    phi = math.radians(lat_deg)
    cos_zen = math.sin(phi) * math.sin(delta) + math.cos(phi) * math.cos(delta) * math.cos(H)
    cos_zen = max(-1.0, min(1.0, cos_zen))
    return math.degrees(math.acos(cos_zen))


# ===================== Polygon Metric (Δ(max−P25)) ====================

def _polygon_values(raster_path: Path, polygon_geom):
    """
    Return all pixel values inside polygon as a 1D array (finite only).
    Reprojects polygon to raster CRS if needed.
    """
    if raster_path is None:
        return np.array([], dtype=np.float32)
    with rasterio.open(raster_path) as ds:
        arr = ds.read(1).astype(np.float32)
        # Reproject polygon → raster CRS if necessary
        if ds.crs and ds.crs.to_epsg() != 4326:
            transformer = Transformer.from_crs(PJCRS.from_epsg(4326), ds.crs, always_xy=True)
            geom = shp_transform(lambda x, y, z=None: transformer.transform(x, y), polygon_geom)
        else:
            geom = polygon_geom
        mask = geometry_mask([mapping(geom)], out_shape=arr.shape, transform=ds.transform, invert=True)
        vals = arr[mask]
        return vals[np.isfinite(vals)]


def poly_delta_max_p25(raster_path: Path, polygon_geom) -> float:
    """
    Compute normalized Δ(max−P25) for pixel radiances within a polygon.

    NOTE: Preserves existing behavior — returns normalized value in [0,1]:
          (max - P25) / (max + P25). Returns NaN if no valid pixels.
    """
    v = _polygon_values(raster_path, polygon_geom)
    if v.size == 0:
        return np.nan
    p25 = np.nanpercentile(v, 25)
    vmax = np.nanmax(v)
    return float((vmax - p25) / (vmax + p25))


# ================================ Main ================================

def main() -> None:
    # ---- Gather inputs (I04 granules & polygons) ----
    recs = gather_i04_index()
    if not recs:
        raise SystemExit("No I04 Radiance granules found in the given range.")
    poly_list = read_polygons_wgs84(POLY_SHP)  # (id, geom, lat, lon)
    print(f"[INFO] Found {len(poly_list)} polygons and {len(recs)} I04 timestamps.")

    # Per-polygon containers (UTC CSV + local plotting series)
    series: Dict[int, Dict[str, List[Any]]] = {
        pid: {
            # CSV (UTC, union day+night)
            "dt": [], "period": [], "delta": [],
            # Plotting (local timezone, separate series)
            "day_dt_loc": [], "day_delta": [],
            "ngt_dt_loc": [], "ngt_delta": [],
        }
        for pid, _, _, _ in poly_list
    }

    # ---- Iterate timestamps; compute deltas; split day/night ----
    for rec in recs:
        dt_utc: datetime = rec["dt"]   # aware UTC
        p04: Path = rec["i04"]
        for pid, geom, plat, plon in poly_list:
            sza = solar_zenith_deg(plat, plon, dt_utc)
            if not np.isfinite(sza):
                continue
            d04 = poly_delta_max_p25(p04, geom) if p04 else np.nan
            if not np.isfinite(d04):
                continue

            period = "day" if sza < SZA_DAY_THRESHOLD_DEG else "night"

            # CSV (UTC)
            series[pid]["dt"].append(dt_utc)
            series[pid]["period"].append(period)
            series[pid]["delta"].append(d04)

            # Plotting (local)
            dt_loc = dt_utc.astimezone(LOCAL_TZ)
            if period == "day":
                series[pid]["day_dt_loc"].append(dt_loc)
                series[pid]["day_delta"].append(d04)
            else:
                series[pid]["ngt_dt_loc"].append(dt_loc)
                series[pid]["ngt_delta"].append(d04)

    # ---- Prepare daily temperature bars from global df_daily (if provided) ----
    def prepare_daily_bars() -> Tuple[List[datetime] | None, np.ndarray | None]:
        """
        Build bar positions (local-noon datetimes) and values (°C) from a global DataFrame `df_daily`
        with columns {'date', 'Ta_C'}. If absent or invalid, returns (None, None).
        """
        if "df_daily" not in globals():
            return None, None
        df = globals()["df_daily"]
        if not isinstance(df, pd.DataFrame) or not {"date", "Ta_C"}.issubset(df.columns):
            return None, None

        # Parse and localize to plotting TZ
        date_ser = pd.to_datetime(df["date"], errors="coerce")
        if date_ser.isna().all():
            return None, None
        if date_ser.dt.tz is None:
            date_ser = date_ser.dt.tz_localize(LOCAL_TZ)
        else:
            date_ser = date_ser.dt.tz_convert(LOCAL_TZ)

        # Center bars at local noon for each date
        noon_ser = date_ser.dt.normalize() + pd.Timedelta(hours=12)
        bar_times = noon_ser.dt.to_pydatetime().tolist()

        # Clean temperature values
        ta = pd.to_numeric(df["Ta_C"], errors="coerce").astype(float)

        # Keep finite temps and valid datetimes
        mask = np.isfinite(ta.values) & pd.notna(noon_ser.values)
        bar_times = [bar_times[i] for i in range(len(bar_times)) if mask[i]]
        bar_vals = ta.values[mask]
        return bar_times, bar_vals

    bar_times, bar_vals = prepare_daily_bars()

    # ===================== CSV export + plotting =======================
    for pid, data in series.items():
        if (not data["day_dt_loc"]) and (not data["ngt_dt_loc"]):
            print(f"[WARN] Polygon id={pid}: no valid samples.")
            continue

        # ---- CSV (UTC), sorted by datetime ----
        order = np.argsort(data["dt"])
        dts_utc = [data["dt"][i] for i in order]
        per     = [data["period"][i] for i in order]
        delt    = [data["delta"][i] for i in order]

        csv_path = OUT_CSV / f"poly_{pid:03d}_timeseries_deltaP25_I04_day_night.csv"
        with csv_path.open("w", newline="") as f:
            w = csv.writer(f)
            w.writerow(["datetime_utc", "period", "delta_I04_Wm2_sr_um"])  # Δ(max−P25), normalized in this script
            for t, p, x in zip(dts_utc, per, delt):
                xs = "" if not np.isfinite(x) else f"{x:.6f}"
                w.writerow([t.strftime("%Y-%m-%dT%H:%MZ"), p, xs])
        print(f"[OK] {csv_path}")

        # ---- Build plotting series (local), each sorted independently ----
        day_t: List[datetime] = []
        day_v: List[float] = []
        ngt_t: List[datetime] = []
        ngt_v: List[float] = []

        if data["day_dt_loc"]:
            d_ord = np.argsort(data["day_dt_loc"])
            day_t = [data["day_dt_loc"][i] for i in d_ord]
            day_v = [data["day_delta"][i]  for i in d_ord]

        if data["ngt_dt_loc"]:
            n_ord = np.argsort(data["ngt_dt_loc"])
            ngt_t = [data["ngt_dt_loc"][i] for i in n_ord]
            ngt_v = [data["ngt_delta"][i]  for i in n_ord]

        # ---- Plot: lines (day/night) + optional Ta_C bars ----
        fig, ax = plt.subplots(figsize=(11, 4.8))

        # Background Ta_C bars on secondary y-axis (if provided)
        ax2 = None
        if bar_times is not None and bar_vals is not None and len(bar_times) > 0:
            ax2 = ax.twinx()
            # Ensure lines render above bars
            ax.set_zorder(2); ax.patch.set_alpha(0.0)
            ax2.set_zorder(1)
            ax2.bar(bar_times, bar_vals, width=0.8, alpha=0.25, color="gray",
                    label="Ta (°C)", zorder=0, align="center")
            ax2.set_ylabel("Air temperature (°C)")
            ax2.set_ylim(20, 35)  # keep your current fixed range
            ax2.grid(False)

        # Day series (solid red, circle markers)
        if day_t:
            ax.plot(day_t, day_v, marker="o", linewidth=1.6, color="red", label="Day", zorder=3)
            for t, v in zip(day_t, day_v):
                ax.annotate(t.strftime("%H:%M"), (t, v), textcoords="offset points",
                            xytext=(0, 6), ha="center", fontsize=8, color="black")

        # Night series (blue dashed, square markers)
        if ngt_t:
            ax.plot(ngt_t, ngt_v, marker="s", linestyle="--", linewidth=1.6, color="blue", label="Night", zorder=3)
            for t, v in zip(ngt_t, ngt_v):
                ax.annotate(t.strftime("%H:%M"), (t, v), textcoords="offset points",
                            xytext=(0, -10), ha="center", fontsize=8, color="black")

        # Fixed y-axis for normalized metric
        ax.set_ylim(0, 1)
        ax.set_xlabel(f"Local time ({LOCAL_TZNAME})")
        ax.set_ylabel(r"Normalized $\Delta(\max - P25)$ (−)")
        ax.grid(True, linestyle="--", alpha=0.4)
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M", tz=LOCAL_TZ))
        fig.autofmt_xdate()

        # Combined legend (include Ta bars if present)
        handles1, labels1 = ax.get_legend_handles_labels()
        if ax2 is not None:
            handles2, labels2 = ax2.get_legend_handles_labels()
            ax.legend(handles1 + handles2, labels1 + labels2, loc="upper left")
        else:
            ax.legend(loc="upper left")

        plt.title(f"Polygon id={pid} — VIIRS I04 Normalized Δ(max−P25)")
        plt.tight_layout()
        plt.show()


# ============================== main =================================

if __name__ == "__main__":
    main()


In [ ]:
# ======================= modis_timeseries.py =======================
"""
Per-polygon time series of MODIS B21 Radiance Δ(max − P25), normalized,
with DAY & NIGHT lines and optional daily Ta_C bars from a global `df_daily` from air_temperature.py.

Metric per polygon & timestamp
------------------------------
  Δ_norm = (max(R) − P25(R)) / (max(R) + P25(R))     ∈ [0, 1]
  R      = pixel radiances within polygon (NaN-aware)
  P25    = 25th percentile via np.nanpercentile(..., 25)

Inputs searched: BASEDIR / YYYY-MM-DD / bt / *_B21_Rad.tif
Outputs (per polygon)
---------------------
  • CSV (UTC): datetime_utc, period{day|night}, delta_norm_B21
  • Inline plot (local time): Day=red solid, Night=blue dashed; y-lim [0, 1]
  • Optional bars (secondary y-axis): daily Ta_C from global `df_daily` (columns: date, Ta_C)

Dependencies
------------
Install numpy pandas rasterio fiona shapely pyproj matplotlib tzdata
"""

from __future__ import annotations

# ============================== Imports ===============================
import re
import csv
import math
from pathlib import Path
from datetime import date, datetime, timedelta, timezone
from typing import List, Tuple, Dict, Any

import numpy as np
import pandas as pd
import rasterio
from rasterio.features import geometry_mask
from shapely.geometry import shape, mapping
from shapely.ops import transform as shp_transform
import fiona
from pyproj import CRS as PJCRS, Transformer

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from zoneinfo import ZoneInfo


# =========================== Configuration ============================

# Root folder containing dated subfolders: YYYY-MM-DD/bt/*.tif
BASEDIR: Path = Path(
    r"path\to\base\directory"
)

# Date range (inclusive)
START_DATE: date = date(2025, 1, 4)
END_DATE:   date = date(2025, 1, 11)

# Plotting timezone (used for line timestamps and x-axis labels)
LOCAL_TZNAME: str     = "America/Los_Angeles" # e.g., "America/Chicago"
LOCAL_TZ: ZoneInfo    = ZoneInfo(LOCAL_TZNAME)

# Day/night threshold (Day if SZA < threshold, otherwise Night)
SZA_DAY_THRESHOLD_DEG: float = 88.0

# Polygons shapefile (must have integer 'id' or 'ID' attribute)
POLY_SHP: Path = Path(
    r"path\to\F002_L1__IR__L2L1M0__2025-01-10T215412.018348Z_2025-04-10T154832.806087Z_97706189_MWIR_Polygon.shp"
)

# Radiance band/pattern (inside each day's "bt" folder)
BAND_LABEL:  str = "B21"
PATTERN_RAD: str = f"*_{BAND_LABEL}_Rad.tif"  # -> "*_B21_Rad.tif"

# Output directories for per-polygon CSVs
OUTDIR: Path  = BASEDIR / "timeseries_modis"
OUT_CSV: Path = OUTDIR / "csv"
OUT_CSV.mkdir(parents=True, exist_ok=True)


# ============================== Utilities =============================

def date_iter(d0: date, d1: date):
    """Inclusive iterator from d0 to d1 (step=1 day)."""
    cur = d0
    while cur <= d1:
        yield cur
        cur += timedelta(days=1)


def timestamp_key_from_name(name: str) -> str | None:
    """Extract MODIS/VIIRS-like timestamp key: '.Ayyyyddd.HHMM.' from filename."""
    m = re.search(r"\.(A\d{7}\.\d{4})\.", name)
    return m.group(1) if m else None


def dt_from_key(key: str) -> datetime | None:
    """Convert 'Ayyyyddd.HHMM' → aware UTC datetime."""
    m = re.match(r"A(\d{4})(\d{3})\.(\d{2})(\d{2})$", key)
    if not m:
        return None
    year, doy, hh, mm = map(int, m.groups())
    return datetime(year, 1, 1, tzinfo=timezone.utc) + timedelta(days=doy - 1, hours=hh, minutes=mm)


def gather_rad_index() -> List[Dict[str, Any]]:
    """
    Collect timestamps & paths for MODIS Radiance granules in per-day bt/ folders.
    Returns a list of records sorted by UTC datetime: {'dt': datetime, 'rad': Path}.
    """
    recs: Dict[str, Dict[str, Any]] = {}
    for day in date_iter(START_DATE, END_DATE):
        btdir = BASEDIR / day.strftime("%Y-%m-%d") / "bt"
        if not btdir.exists():
            continue
        for p in btdir.glob(PATTERN_RAD):
            key = timestamp_key_from_name(p.name)
            if not key:
                continue
            dt = dt_from_key(key)
            if not dt:
                continue
            recs[key] = {"dt": dt, "rad": p}
    return [recs[k] for k in sorted(recs.keys(), key=lambda kk: recs[kk]["dt"])]


def read_polygons_wgs84(shp_path: Path) -> List[Tuple[int, Any, float, float]]:
    """
    Read polygons and return them in EPSG:4326 with centroid lat/lon.

    Returns
    -------
    list[(poly_id, shapely_geom_4326, centroid_lat, centroid_lon)]
    """
    if not shp_path.exists():
        raise FileNotFoundError(f"Polygon shapefile not found: {shp_path}")

    polys: List[Tuple[int, Any, float, float]] = []
    with fiona.open(shp_path, "r") as src:
        src_crs = src.crs
        if not src_crs:
            raise ValueError("Input shapefile has no CRS.")
        src_crs_obj = PJCRS.from_user_input(src_crs)
        dst_crs_obj = PJCRS.from_epsg(4326)

        if src_crs_obj != dst_crs_obj:
            transformer = Transformer.from_crs(src_crs_obj, dst_crs_obj, always_xy=True)
            reproj = lambda x, y, z=None: transformer.transform(x, y)
        else:
            reproj = None

        for feat in src:
            props = feat.get("properties", {})
            poly_id = int(props.get("id", props.get("ID", len(polys) + 1)))
            geom = shape(feat["geometry"]).buffer(0)  # make valid if needed
            if reproj:
                geom = shp_transform(reproj, geom)
            c = geom.centroid
            polys.append((poly_id, geom, float(c.y), float(c.x)))

    if not polys:
        raise ValueError("No polygons found in shapefile.")
    return polys


# =========================== Solar Geometry ===========================

def solar_zenith_deg(lat_deg: float, lon_deg: float, dt_utc: datetime) -> float:
    """
    Approximate solar zenith angle (degrees) for a given UTC datetime and lat/lon.
    Accuracy is sufficient for day/night classification.
    """
    y, m, d = dt_utc.year, dt_utc.month, dt_utc.day
    hh, mm = dt_utc.hour, dt_utc.minute
    ss = dt_utc.second + dt_utc.microsecond / 1e6

    # Julian Day
    a = math.floor((14 - m) / 12)
    y2 = y + 4800 - a
    m2 = m + 12 * a - 3
    jd = d + math.floor((153 * m2 + 2) / 5) + 365 * y2 + math.floor(y2 / 4) \
         - math.floor(y2 / 100) + math.floor(y2 / 400) - 32045
    frac_day = (hh + mm / 60 + ss / 3600) / 24.0
    JD = jd + frac_day
    T = (JD - 2451545.0) / 36525.0

    # Solar coordinates
    L0 = (280.46646 + 36000.76983 * T + 0.0003032 * T * T) % 360
    M = math.radians((357.52911 + 35999.05029 * T - 0.0001537 * T * T) % 360)
    e = 0.016708634 - 0.000042037 * T - 0.0000001267 * T * T
    C = (1.914602 - 0.004817 * T - 0.000014 * T * T) * math.sin(M) + \
        (0.019993 - 0.000101 * T) * math.sin(2 * M) + 0.000289 * math.sin(3 * M)
    lam_true = math.radians((L0 + C) % 360)
    omega = math.radians(125.04 - 1934.136 * T)
    lam_app = lam_true - math.radians(0.00569) - math.radians(0.00478) * math.sin(omega)

    eps0 = 23 + (26 + (21.448 - T * (46.815 + T * (0.00059 - 0.001813 * T))) / 60) / 60
    eps = math.radians(eps0 + 0.00256 * math.cos(omega))

    # Declination
    sin_delta = math.sin(eps) * math.sin(lam_app)
    delta = math.asin(sin_delta)

    # Equation of Time (minutes)
    y_eps = math.tan(eps / 2) ** 2
    L0r = math.radians(L0)
    E = 4 * math.degrees(
        y_eps * math.sin(2 * L0r) - 2 * e * math.sin(M) + 4 * e * y_eps * math.sin(M) * math.cos(2 * L0r)
        - 0.5 * y_eps * y_eps * math.sin(4 * L0r) - 1.25 * e * e * math.sin(2 * M)
    )

    # True Solar Time (minutes) & hour angle
    utc_min = hh * 60 + mm + ss / 60
    TST = (utc_min + E + 4 * lon_deg) % 1440
    H = math.radians(TST / 4 - 180)

    # Zenith angle
    phi = math.radians(lat_deg)
    cos_zen = math.sin(phi) * math.sin(delta) + math.cos(phi) * math.cos(delta) * math.cos(H)
    cos_zen = max(-1.0, min(1.0, cos_zen))
    return math.degrees(math.acos(cos_zen))


# ==================== Polygon Metric: Δ_norm ==========================

def _polygon_values(raster_path: Path, polygon_geom):
    """
    Return all finite pixel values inside polygon as 1D array.
    Reprojects polygon → raster CRS if necessary.
    """
    if raster_path is None:
        return np.array([], dtype=np.float32)
    with rasterio.open(raster_path) as ds:
        arr = ds.read(1).astype(np.float32)
        if ds.crs and ds.crs.to_epsg() != 4326:
            transformer = Transformer.from_crs(PJCRS.from_epsg(4326), ds.crs, always_xy=True)
            geom = shp_transform(lambda x, y, z=None: transformer.transform(x, y), polygon_geom)
        else:
            geom = polygon_geom
        mask = geometry_mask([mapping(geom)], out_shape=arr.shape, transform=ds.transform, invert=True)
        vals = arr[mask]
        return vals[np.isfinite(vals)]


def poly_delta_norm(raster_path: Path, polygon_geom) -> float:
    """
    Δ_norm = (max − P25) / (max + P25)
    Returns NaN if no valid pixels or denominator ≤ 0.
    """
    v = _polygon_values(raster_path, polygon_geom)
    if v.size == 0:
        return np.nan
    vmax = np.nanmax(v)
    p25  = np.nanpercentile(v, 25)
    denom = vmax + p25
    if not np.isfinite(denom) or denom <= 0:
        return np.nan
    return float((vmax - p25) / denom)


# ================================ Main ================================

def main() -> None:
    # ---- Gather inputs ----
    recs = gather_rad_index()
    if not recs:
        raise SystemExit(f"No MODIS {BAND_LABEL} Radiance granules found in the given range.")
    poly_list = read_polygons_wgs84(POLY_SHP)  # (id, geom, lat, lon)
    print(f"[INFO] Found {len(poly_list)} polygons and {len(recs)} timestamps.")

    # Per-polygon storage (UTC CSV + local plotting series)
    series: Dict[int, Dict[str, List[Any]]] = {
        pid: {
            "dt": [], "period": [], "delta": [],
            "day_dt_loc": [], "day_delta": [],
            "ngt_dt_loc": [], "ngt_delta": [],
        } for pid, _, _, _ in poly_list
    }

    # ---- Iterate timestamps; compute Δ_norm; split day/night ----
    for rec in recs:
        dt_utc: datetime = rec["dt"]  # aware UTC
        p_rad: Path      = rec["rad"]
        for pid, geom, plat, plon in poly_list:
            sza = solar_zenith_deg(plat, plon, dt_utc)
            if not np.isfinite(sza):
                continue

            dval = poly_delta_norm(p_rad, geom) if p_rad else np.nan
            if not np.isfinite(dval):
                continue

            period = "day" if sza < SZA_DAY_THRESHOLD_DEG else "night"

            # CSV (UTC)
            series[pid]["dt"].append(dt_utc)
            series[pid]["period"].append(period)
            series[pid]["delta"].append(dval)

            # Plotting (local)
            dt_loc = dt_utc.astimezone(LOCAL_TZ)
            if period == "day":
                series[pid]["day_dt_loc"].append(dt_loc)
                series[pid]["day_delta"].append(dval)
            else:
                series[pid]["ngt_dt_loc"].append(dt_loc)
                series[pid]["ngt_delta"].append(dval)

    # ---- Optional: daily temperature bars from global df_daily ----
    def prepare_daily_bars():
        """
        Return (bar_times_local_noon, bar_vals) from a global `df_daily` with
        columns {'date', 'Ta_C'}. If absent/invalid → (None, None).
        """
        if "df_daily" not in globals():
            return None, None
        df = globals()["df_daily"]
        if not isinstance(df, pd.DataFrame) or not {"date", "Ta_C"}.issubset(df.columns):
            return None, None

        date_ser = pd.to_datetime(df["date"], errors="coerce")
        if date_ser.isna().all():
            return None, None
        if date_ser.dt.tz is None:
            date_ser = date_ser.dt.tz_localize(LOCAL_TZ)
        else:
            date_ser = date_ser.dt.tz_convert(LOCAL_TZ)

        noon_ser = date_ser.dt.normalize() + pd.Timedelta(hours=12)
        t_mid = noon_ser.dt.to_pydatetime().tolist()
        ta = pd.to_numeric(df["Ta_C"], errors="coerce").astype(float).values
        mask = np.isfinite(ta) & pd.notna(noon_ser).values
        bar_times = [t_mid[i] for i in range(len(t_mid)) if mask[i]]
        bar_vals  = ta[mask]
        return bar_times, bar_vals

    bar_times, bar_vals = prepare_daily_bars()

    # ===================== CSV export + plotting ======================
    for pid, data in series.items():
        if (not data["day_dt_loc"]) and (not data["ngt_dt_loc"]):
            print(f"[WARN] Polygon id={pid}: no valid samples.")
            continue

        # ---- CSV (UTC), sorted by datetime ----
        order = np.argsort(data["dt"])
        dts_utc = [data["dt"][i]     for i in order]
        per     = [data["period"][i] for i in order]
        delt    = [data["delta"][i]  for i in order]

        csv_path = OUT_CSV / f"poly_{pid:03d}_timeseries_deltaNorm_{BAND_LABEL}_day_night.csv"
        with csv_path.open("w", newline="") as f:
            w = csv.writer(f)
            w.writerow(["datetime_utc", "period", f"delta_norm_{BAND_LABEL}"])
            for t, p, x in zip(dts_utc, per, delt):
                xs = "" if not np.isfinite(x) else f"{x:.6f}"
                w.writerow([t.strftime("%Y-%m-%dT%H:%MZ"), p, xs])
        print(f"[OK] {csv_path}")

        # ---- Build plotting series (local), each sorted independently ----
        day_t: List[datetime] = []
        day_v: List[float] = []
        ngt_t: List[datetime] = []
        ngt_v: List[float] = []

        if data["day_dt_loc"]:
            d_ord = np.argsort(data["day_dt_loc"])
            day_t = [data["day_dt_loc"][i] for i in d_ord]
            day_v = [data["day_delta"][i]  for i in d_ord]

        if data["ngt_dt_loc"]:
            n_ord = np.argsort(data["ngt_dt_loc"])
            ngt_t = [data["ngt_dt_loc"][i] for i in n_ord]
            ngt_v = [data["ngt_delta"][i]  for i in n_ord]

        # ---- Plot: day/night lines + optional Ta_C bars ----
        fig, ax = plt.subplots(figsize=(11, 4.8))

        ax2 = None
        if bar_times is not None and bar_vals is not None and len(bar_times) > 0:
            ax2 = ax.twinx()
            ax.set_zorder(2); ax.patch.set_alpha(0.0)
            ax2.set_zorder(1)
            ax2.bar(bar_times, bar_vals, width=0.8, alpha=0.25, color="gray",
                    label="Ta (°C)", zorder=0, align="center")
            ax2.set_ylabel("Air temperature (°C)")
            ax2.grid(False)

        # Day series (solid red)
        if day_t:
            ax.plot(day_t, day_v, marker="o", linewidth=1.6, color="red", label="Day")
            for t, v in zip(day_t, day_v):
                ax.annotate(t.strftime("%H:%M"), (t, v), textcoords="offset points",
                            xytext=(0, 6), ha="center", fontsize=8, color="black")

        # Night series (blue dashed)
        if ngt_t:
            ax.plot(ngt_t, ngt_v, marker="s", linestyle="--", linewidth=1.6, color="blue", label="Night")
            for t, v in zip(ngt_t, ngt_v):
                ax.annotate(t.strftime("%H:%M"), (t, v), textcoords="offset points",
                            xytext=(0, -10), ha="center", fontsize=8, color="black")

        # Axes/labels/legend
        ax.set_ylim(0.0, 1.0)  # normalized metric
        ax.set_xlabel(f"Local time ({LOCAL_TZNAME}) — day if SZA < {SZA_DAY_THRESHOLD_DEG}°")
        ax.set_ylabel(r"Normalized Radiance (−)")
        ax.grid(True, linestyle="--", alpha=0.4)
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M", tz=LOCAL_TZ))
        fig.autofmt_xdate()

        handles1, labels1 = ax.get_legend_handles_labels()
        if ax2 is not None:
            handles2, labels2 = ax2.get_legend_handles_labels()
            ax.legend(handles1 + handles2, labels1 + labels2, loc="upper left")
        else:
            ax.legend(loc="upper left")

        plt.title(f"Polygon id={pid} — MODIS {BAND_LABEL} Normalized Δ(max−P25)")
        plt.tight_layout()
        plt.show()


# ============================== main =================================

if __name__ == "__main__":
    main()


In [ ]:
# ========================== false_positive.py =================================
"""
OroraTech False Positives vs Reference (VIIRS or MODIS)
----------------------------------------------------------------------------
Behavior
  • Builds a false-positive (FP) table using a 7×7 kernel tested against reference
    detection rasters.
  • Skips rasters with no/partial coverage (requires ALL OroraTech points covered).
  • Sorts by local time; deduplicates by path/name/time; exports a public subset to Excel.
  • Plots, per row:
      - Esri World Imagery basemap
      - Enlarged yellow rectangles at detected pixels (>= threshold), black edges
      - Points: FP=red (white edge), TP=green (white edge)
      - Scalebar (meters)
      - Zoom framing via ZOOM_OUT_FACTOR + MIN_HALF_SPAN_M

Dependencies
  Install numpy pandas rasterio fiona shapely pyproj matplotlib tzdata
          contextily xyzservices matplotlib-scalebar

"""
# =============================================================================

from __future__ import annotations

# ============================== Imports ======================================
import re
from pathlib import Path
from datetime import datetime, date, timedelta, timezone
from typing import List, Optional, Dict, Tuple

import numpy as np
import pandas as pd

import rasterio
from rasterio.windows import Window
from rasterio.crs import CRS as RCRS
from rasterio.transform import xy as rio_xy

import fiona
from shapely.geometry import shape, Point
from pyproj import Transformer, CRS as PJCRS
from zoneinfo import ZoneInfo

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

from IPython.display import display

import contextily as ctx
from matplotlib_scalebar.scalebar import ScaleBar


# ========================= User Configuration ================================

# --- Satellite & temporal window (inclusive dates as 'YYYY-MM-DD') ---
SATELLITE   = "MODIS" # "VIIRS" or "MODIS"
START_DATE  = "2025-01-10"
END_DATE    = "2025-01-11"

# --- Paths (toggle the pair you need) ---
REF_BASEDIR   = Path(r"path\to\satellite\reference\directory")
ORORA_GEOJSON = Path(r"path\to\F002_L1__IR__L2L1M0__2025-01-10T215412.018348Z_2025-04-10T154832.806087Z_97706189_model_detections.geojson")
out_path      = Path(r"save\to\ororatech_detection_results.xlsx")

# --- Detection & geometry parameters ---
KERNEL_SIZE       = 7          # size of square kernel (K×K) used for reference detection check
DETECT_THRESHOLD  = 0.5        # pixel >= threshold ⇒ detected
INPUT_POINTS_CRS  = PJCRS.from_epsg(4326)

# --- Local timezone for reporting/sorting ---
# LOCAL_TZ = ZoneInfo("America/Chicago")   # Texas
LOCAL_TZ = ZoneInfo("America/Los_Angeles") # California

# --- Display & drawing parameters (EPSG:3857) ---
PIXEL_RECT_SCALE  = 15.0       # rectangle scale relative to pixel size (in meters via 3857)
MAX_RECT_PATCHES  = 15000      # thin rectangles if detections exceed this cap
ZOOM_PAD_M        = 5000.0     # padding meters on each side
ZOOM_OUT_FACTOR   = 1.4        # inflate half-width/height by this factor
MIN_HALF_SPAN_M   = 8000.0     # min half-span (meters) to avoid over-zooming


# =============================== Utilities ===================================

def _date_iter(d0: date, d1: date):
    """Inclusive date iterator from d0 to d1 (step = 1 day)."""
    cur = d0
    while cur <= d1:
        yield cur
        cur += timedelta(days=1)


def _suffix_for_sat(sat: str) -> str:
    """
    Expected suffix for reference detection rasters by satellite.
    VIIRS → *_detectmask.tif, MODIS → *_detect.tif
    """
    s = sat.upper()
    if s == "VIIRS":
        return "_detectmask.tif"
    if s == "MODIS":
        return "_detect.tif"
    raise ValueError("SATELLITE must be 'VIIRS' or 'MODIS'")


def _gather_reference_files_for_date(day: date, sat: str) -> List[Path]:
    """
    Find all detection rasters for a given date under .../<YYYY-MM-DD>/af/.
    Only files with the satellite-specific suffix are returned.
    """
    daydir = REF_BASEDIR / day.strftime("%Y-%m-%d") / "af"
    if not daydir.exists():
        return []
    suf = _suffix_for_sat(sat)
    found = set()
    for p in daydir.rglob("*.tif"):
        if p.name.lower().endswith(suf):
            try:
                found.add(p.resolve())
            except Exception:
                found.add(p)
    return list(found)


def _parse_dt_utc_from_name(name: str) -> Optional[datetime]:
    """
    Parse UTC from filenames containing '.Ayyyyddd.HHMM.' (e.g., MODIS/VIIRS convention).
    Returns an aware UTC datetime or None if not found.
    """
    m = re.search(r"\.A(\d{4})(\d{3})\.(\d{2})(\d{2})\.", name)
    if not m:
        return None
    year, doy, hh, mm = map(int, m.groups())
    return datetime(year, 1, 1, tzinfo=timezone.utc) + timedelta(days=doy - 1, hours=hh, minutes=mm)


def _count_reference_detections(raster_path: Path) -> int:
    """Count pixels meeting the detection threshold (>= DETECT_THRESHOLD)."""
    with rasterio.open(raster_path) as ds:
        band = ds.read(1, masked=True).filled(np.nan)
        return int(np.nansum(band >= DETECT_THRESHOLD))


def _load_orora_points(geojson_path: Path) -> List[Point]:
    """
    Load OroraTech detections as shapely Points from a GeoJSON.
    Only Point geometries are considered (multipoints ignored).
    """
    if not geojson_path.exists():
        raise FileNotFoundError(f"OroraTech GeoJSON not found: {geojson_path}")
    pts: List[Point] = []
    with fiona.open(geojson_path, "r") as src:
        for feat in src:
            g = shape(feat["geometry"])
            if isinstance(g, Point):
                pts.append(g)
    if not pts:
        print("[WARN] No point detections found in the GeoJSON.")
    return pts


def _point_fp_mask_and_coverage(
    raster_path: Path, pts: List[Point], kernel: int
) -> Tuple[np.ndarray, np.ndarray]:
    """
    For each OroraTech point, test a K×K neighborhood in the reference raster.

    Returns
    -------
    fp_mask : bool array
        True  → False Positive (no detection in K×K neighborhood).
        False → True Positive  (≥ threshold exists in K×K neighborhood).
    covered_mask : bool array
        True if the point is fully covered by the raster & the read window is valid.

    Notes
    -----
    • Points not covered (covered_mask=False) are not counted toward FP/TP.
    • The caller enforces "all points must be covered"; otherwise that raster is skipped.
    """
    half = kernel // 2
    fp_mask = np.zeros(len(pts), dtype=bool)
    covered = np.zeros(len(pts), dtype=bool)

    with rasterio.open(raster_path) as ds:
        ds_crs = ds.crs or RCRS.from_epsg(4326)
        tf: Optional[Transformer] = None
        # Build a transformer only if raster CRS differs from input points CRS
        try:
            if (ds_crs.to_epsg() or 4326) != 4326:
                tf = Transformer.from_crs(INPUT_POINTS_CRS, PJCRS.from_wkt(ds_crs.to_wkt()), always_xy=True)
        except Exception:
            # Fall back to assuming same CRS if EPSG resolution fails
            tf = None

        for i, pt in enumerate(pts):
            x, y = (tf.transform(pt.x, pt.y) if tf else (pt.x, pt.y))
            try:
                row, col = ds.index(x, y)
            except Exception:
                covered[i] = False
                continue

            r0 = max(0, row - half); r1 = min(ds.height, row + half + 1)
            c0 = max(0, col - half); c1 = min(ds.width,  col + half + 1)
            if (r1 <= r0) or (c1 <= c0):
                covered[i] = False
                continue

            win = Window.from_slices((r0, r1), (c0, c1))
            arr = ds.read(1, window=win, boundless=False, masked=True).filled(np.nan)
            if not np.isfinite(arr).any():
                covered[i] = False
                continue

            covered[i] = True
            det = (np.nanmax(arr) >= DETECT_THRESHOLD)
            fp_mask[i] = (not det)

    return fp_mask, covered


# =============================== Plotting =====================================

def _plot_overlay_detect_mask_merc_esri(
    raster_path: Path,
    pts: List[Point],
    fp_mask: np.ndarray,
    title: str,
    zoom_to_points: bool = True,
    pad_m: float = ZOOM_PAD_M,
) -> None:
    """
    Plot detection rectangles and FP/TP points on an Esri World Imagery basemap in EPSG:3857.

    Layers (z-order)
      • Basemap (contextily, EPSG:3857)
      • Enlarged yellow rectangles centered on detected pixels (black edges)
      • FP points (red, white edge), TP points (green, white edge)
      • Scalebar (meters), legend, title

    Framing
      • Either fit to all points or to raster bounds, then expand by:
          hx, hy = max(halfspan * ZOOM_OUT_FACTOR + pad, MIN_HALF_SPAN_M)
    """
    EPSG3857 = PJCRS.from_epsg(3857)

    with rasterio.open(raster_path) as ds:
        ds_crs = ds.crs or RCRS.from_epsg(4326)
        to_merc = Transformer.from_crs(PJCRS.from_wkt(ds_crs.to_wkt()) if ds_crs else PJCRS.from_epsg(4326),
                                       EPSG3857, always_xy=True)
        from_wgs84_to_merc = Transformer.from_crs(PJCRS.from_epsg(4326), EPSG3857, always_xy=True)

        # --- Detection mask ---
        arr = ds.read(1).astype(np.float32)
        det = (arr >= DETECT_THRESHOLD) & np.isfinite(arr)

        # --- Raster extent → 3857 ---
        left, bottom, right, top = ds.bounds
        bx0, by0 = to_merc.transform(left,  bottom)
        bx1, by1 = to_merc.transform(right, top)
        x_min, x_max = min(bx0, bx1), max(bx0, bx1)
        y_min, y_max = min(by0, by1), max(by0, by1)

        # --- Init figure/axes (no ticks/labels) ---
        fig, ax = plt.subplots(figsize=(10, 8.5))
        ax.set_xlim(x_min, x_max); ax.set_ylim(y_min, y_max)
        ax.set_xticks([]); ax.set_yticks([])
        ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

        # --- Basemap (Esri World Imagery) ---
        ctx.add_basemap(ax, crs="EPSG:3857", source=ctx.providers.Esri.WorldImagery)

        # --- Detection rectangles (thin if too many) ---
        rr, cc = np.where(det)
        n = rr.size
        if n > 0:
            # Pixel metric size estimated near scene center (in 3857)
            px_dx = abs(ds.transform.a); px_dy = abs(ds.transform.e)
            cx_src = (left + right) / 2.0; cy_src = (bottom + top) / 2.0
            cxm, cym = to_merc.transform(cx_src, cy_src)
            dxm, _   = to_merc.transform(cx_src + px_dx, cy_src)
            _,   dym = to_merc.transform(cx_src, cy_src + px_dy)
            px_w_m = abs(dxm - cxm); px_h_m = abs(dym - cym)
            w = px_w_m * float(PIXEL_RECT_SCALE)
            h = px_h_m * float(PIXEL_RECT_SCALE)

            stride = 1
            if n > MAX_RECT_PATCHES:
                stride = int(np.ceil(np.sqrt(n / MAX_RECT_PATCHES)))
            rr = rr[::stride]; cc = cc[::stride]

            xs_src, ys_src = rio_xy(ds.transform, rr, cc, offset="center")
            xs_m, ys_m = to_merc.transform(xs_src, ys_src)

            patches = [Rectangle((x - w/2, y - h/2), w, h) for x, y in zip(xs_m, ys_m)]
            coll = PatchCollection(patches, facecolor="yellow", edgecolor="black", linewidth=1, zorder=2)
            ax.add_collection(coll)

        # --- Points (WGS84 → 3857) with white borders ---
        xs_fp: List[float]; ys_fp: List[float]; xs_tp: List[float]; ys_tp: List[float]
        xs_fp, ys_fp, xs_tp, ys_tp, xs_all, ys_all = [], [], [], [], [], []
        for is_fp, p in zip(fp_mask, pts):
            xm, ym = from_wgs84_to_merc.transform(p.x, p.y)
            xs_all.append(xm); ys_all.append(ym)
            if is_fp: xs_fp.append(xm); ys_fp.append(ym)
            else:     xs_tp.append(xm); ys_tp.append(ym)

        if xs_fp:
            ax.scatter(xs_fp, ys_fp, s=150, c="red",   edgecolors="white", linewidths=1.1,
                       label="False Positive", zorder=5)
        if xs_tp:
            ax.scatter(xs_tp, ys_tp, s=150, c="green", edgecolors="white", linewidths=1.1,
                       label="True Positive",  zorder=6)

        # --- Framing / zoom ---
        if zoom_to_points and len(xs_all) > 0:
            x0, x1 = min(xs_all), max(xs_all)
            y0, y1 = min(ys_all), max(ys_all)
        else:
            x0, x1 = x_min, x_max; y0, y1 = y_min, y_max

        xc = 0.5 * (x0 + x1); yc = 0.5 * (y0 + y1)
        hx = max(0.5 * (x1 - x0), 1.0); hy = max(0.5 * (y1 - y0), 1.0)
        hx = max(hx * ZOOM_OUT_FACTOR + pad_m, MIN_HALF_SPAN_M)
        hy = max(hy * ZOOM_OUT_FACTOR + pad_m, MIN_HALF_SPAN_M)
        ax.set_xlim(xc - hx, xc + hx); ax.set_ylim(yc - hy, yc + hy)

        # --- Title, legend, scalebar ---
        ax.set_title(title, fontsize=26, fontweight="bold")
        ax.legend(loc="lower right", frameon=True, fontsize=18, title_fontsize=15)
        try:
            scalebar = ScaleBar(dx=1, units='m', location='upper right', box_alpha=0.6, scale_loc='top')
            ax.add_artist(scalebar)
        except Exception as e:
            print(f"[WARN] Could not add scalebar: {e}")

        plt.tight_layout()
        plt.show()


# ============================== Core Pipeline ================================

def _build_table_and_masks(
    SATELLITE: str,
    START_DATE: str,
    END_DATE: str,
) -> tuple[pd.DataFrame, List[Point], Dict[str, np.ndarray]]:
    """
    Build the false-positive summary table across the requested date window and
    compute per-raster FP masks (for plotting), enforcing full coverage.

    Returns
    -------
    df : pandas.DataFrame
        Full table including hidden plotting columns (paths, datetimes).
    orora_pts : list[Point]
        OroraTech points as shapely Points (WGS84).
    FP_MASK_BY_PATH : dict[str, np.ndarray]
        Map from resolved raster path → pointwise FP mask (bool array).
    """
    D0 = datetime.strptime(START_DATE, "%Y-%m-%d").date()
    D1 = datetime.strptime(END_DATE,   "%Y-%m-%d").date()
    if D1 < D0:
        raise SystemExit("END_DATE must be >= START_DATE")

    orora_pts = _load_orora_points(ORORA_GEOJSON)
    N_POINTS = len(orora_pts)
    print(f"[INFO] Loaded {N_POINTS} OroraTech detections from {ORORA_GEOJSON.name}")
    print(f"[INFO] Local timezone for reporting: {getattr(LOCAL_TZ, 'key', None) or LOCAL_TZ.tzname(datetime.now())}")

    rows: List[dict] = []
    FP_MASK_BY_PATH: Dict[str, np.ndarray] = {}

    # ---- Enumerate reference rasters by day ----
    for day in _date_iter(D0, D1):
        ref_paths = _gather_reference_files_for_date(day, SATELLITE)
        if not ref_paths:
            continue

        # Sort: by timestamp (if parseable), then name
        ref_paths = sorted(
            ref_paths,
            key=lambda p: (
                (dt := _parse_dt_utc_from_name(p.name)) is None,
                (dt.strftime("%H%M") if dt else ""),
                p.name,
            ),
        )

        for rp in ref_paths:
            dt_utc = _parse_dt_utc_from_name(rp.name) or datetime.combine(day, datetime.min.time(), tzinfo=timezone.utc)
            dt_loc = dt_utc.astimezone(LOCAL_TZ)

            # Skip unreadable rasters
            try:
                ref_det = _count_reference_detections(rp)
            except Exception:
                continue

            # Build FP mask & coverage; enforce FULL coverage across all points
            fp_mask, covered = _point_fp_mask_and_coverage(rp, orora_pts, KERNEL_SIZE)
            if (covered.size == 0) or (covered.sum() != N_POINTS):
                continue

            fp = int(fp_mask.sum())
            if N_POINTS == 0:
                continue
            fp_rate = fp / N_POINTS if N_POINTS > 0 else np.nan

            key_path = str(rp.resolve())
            FP_MASK_BY_PATH[key_path] = fp_mask

            try:
                raster_rel = str(rp.resolve().relative_to(REF_BASEDIR.resolve()))
            except Exception:
                raster_rel = rp.name

            rows.append({
                "satellite": SATELLITE,
                "local_date": dt_loc.date(),
                "local_time": dt_loc.strftime("%H:%M"),
                "tz": dt_loc.tzname(),
                "false_positives": fp,
                "fp_rate": round(fp_rate, 3) if np.isfinite(fp_rate) else np.nan,
                "ref_detections": int(ref_det),
                "raster_name": rp.name,
                "_raster_path": key_path,     # for plotting
                "_raster_relpath": raster_rel,
                "_local_dt": dt_loc,          # sort key
            })

    df = pd.DataFrame(rows)
    if df.empty:
        print("No rasters with full coverage found in the requested date range.")
        return df, orora_pts, FP_MASK_BY_PATH

    # ---- Deduplicate & sort ----
    df = df.drop_duplicates(subset=["_raster_relpath"], keep="first").copy()
    df = df.sort_values(["_local_dt", "ref_detections"], ascending=[True, False])
    df = df.drop_duplicates(subset=["raster_name", "local_date", "local_time"], keep="first")
    df = df.sort_values("_local_dt").reset_index(drop=True)
    return df, orora_pts, FP_MASK_BY_PATH


# ============================ Execute & Plot ================================

# Build table + masks
df, orora_pts, FP_MASK_BY_PATH = _build_table_and_masks(SATELLITE, START_DATE, END_DATE)

# Export public subset & display; then iterate plots row-by-row
if not df.empty:
    df_public = df[[
        "satellite", "local_date", "local_time", "tz",
        "false_positives", "fp_rate", "ref_detections", "raster_name"
    ]].copy()

    # Export to Excel
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df_public.to_excel(out_path, index=False)
    print(f"[INFO] Results saved to: {out_path.resolve()}")

    # Show the public view inline
    display(df_public)

    # Per-row overlays (Esri basemap; EPSG:3857; framed zoom)
    for _, row in df.iterrows():
        rpath = Path(row["_raster_path"])
        fp_mask = FP_MASK_BY_PATH.get(str(rpath.resolve()))
        if (not rpath.exists()) or (fp_mask is None):
            print(f"[SKIP] Missing raster or FP mask for {row['raster_name']}")
            continue

        title = f"{row['satellite']} Detection — {row['local_date']} {row['local_time']} {row['tz']}"
        _plot_overlay_detect_mask_merc_esri(rpath, orora_pts, fp_mask, title)
